<a href="https://colab.research.google.com/github/jtapiav/github-slideshow/blob/master/Extending_Kernighan_Lin_Partitioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

In [ ]:
import random

#Original Kernighan-Lin Partition

## Graph

In [ ]:
class CoreGraph:
    def __init__(self, cores, edges):
        self.cores = cores
        self.edges = edges

## Calculate Cost

In [ ]:
def calculate_cost(graph, partition1, partition2):
    cost = 0
    for edge in graph.edges:
        if (edge[0] in partition1 and edge[1] in partition2) or (edge[0] in partition2 and edge[1] in partition1):
            cost += 1
    return cost

## Swap Core

In [ ]:
def get_next_move(graph, partition1, partition2):
    best_cost = calculate_cost( graph, partition1, partition2 )
    best_move = None
    for ci in partition1:
        for cj in partition2:
            current_cost = calculate_cost(graph, partition1 - {ci} | {cj}, partition2 - {cj} | {ci})
            if current_cost < best_cost:
                best_cost = current_cost
                best_move = (ci, cj)
    return best_move

## Kernighan_Lin Partition

In [ ]:
def kernighan_lin(graph, partition1, partition2, level=0):
    best_partition1, best_partition2 = partition1.copy(), partition2.copy()
    current_partition1, current_partition2 = partition1.copy(), partition2.copy()

    max_iterations = 50

    for iteration in range(max_iterations):
        next_move = get_next_move(graph, current_partition1, current_partition2)
        if next_move is None:
            break  # Salimos del ciclo si no encontramos una mejor jugada
        ci, cj = next_move
        current_partition1.remove(ci)
        current_partition2.remove(cj)
        current_partition1.add(cj)
        current_partition2.add(ci)

        if calculate_cost(graph, current_partition1, current_partition2) < calculate_cost(graph, best_partition1, best_partition2):
            best_partition1, best_partition2 = current_partition1.copy(), current_partition2.copy()

    if len(best_partition1) == len(partition1) and len(best_partition2) == len(partition2):
        return best_partition1, best_partition2

    p1, p2 = best_partition1, best_partition2
    return p1, p2

## Extend Kernighan-Lin Partition

In [ ]:
def kl_partitioning(graph, level, cores_to_be_partitioned):
  cores_to_be_partitioned = set(cores_to_be_partitioned)  # Convert to set to allow set operations

  if len(cores_to_be_partitioned) < 4:
    return cores_to_be_partitioned

  cores_list = list(cores_to_be_partitioned)

  partition1 = set(random.sample(cores_list, len(cores_list) // 2))
  partition2 = cores_to_be_partitioned - partition1

  indent = " " * (level * 4)
  print(indent + f"Level {level}:")
  print(indent + f"Complete Set: {cores_to_be_partitioned}")
  print(indent + f"Start Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1, partition2 = kernighan_lin(graph, partition1, partition2)

  indent = " " * (level * 4)
  print(indent + f"Final Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1_1, partition1_2 = kl_partitioning(graph, level + 1, partition1)
  partition2_1, partition2_2 = kl_partitioning(graph, level + 1, partition2)

  partition1 = (partition1_1, partition1_2)
  partition2 = (partition2_1, partition2_2)

  return partition1, partition2


## Test

In [ ]:
# Ejemplo de uso
cores = {'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16'}
edges = {('C1', 'C2'), ('C1', 'C3'), ('C2', 'C4'), ('C3', 'C4'), ('C3', 'C5'), ('C4', 'C6'), ('C5', 'C6'),
         ('C7', 'C8'), ('C7', 'C9'), ('C8', 'C10'), ('C9', 'C10'), ('C9', 'C11'), ('C10', 'C12'), ('C11', 'C12'),
         ('C13', 'C14'), ('C13', 'C15'), ('C14', 'C16'), ('C15', 'C16')}
graph = CoreGraph(cores, edges)

partition1, partition2 = kl_partitioning(graph, 0, cores)

print( "Cost:" f"{calculate_cost( graph, partition1, partition2 )}" )
print("Final Partition 1:", partition1)
print("Final Partition 2:", partition2)

Level 0:
Complete Set: {'C15', 'C4', 'C7', 'C10', 'C2', 'C1', 'C13', 'C16', 'C6', 'C14', 'C9', 'C8', 'C3', 'C11', 'C5', 'C12'}
Start Partitions Cost: 10
 Partition 1: {'C13', 'C4', 'C7', 'C16', 'C6', 'C8', 'C10', 'C5'}
 Partition 2: {'C15', 'C14', 'C9', 'C11', 'C3', 'C2', 'C1', 'C12'}
Final Partitions Cost: 4
 Partition 1: {'C4', 'C7', 'C6', 'C9', 'C8', 'C3', 'C10', 'C5'}
 Partition 2: {'C15', 'C16', 'C14', 'C11', 'C2', 'C1', 'C13', 'C12'}
    Level 1:
    Complete Set: {'C4', 'C7', 'C6', 'C9', 'C8', 'C3', 'C10', 'C5'}
    Start Partitions Cost: 4
     Partition 1: {'C4', 'C7', 'C8', 'C6'}
     Partition 2: {'C9', 'C3', 'C5', 'C10'}
    Final Partitions Cost: 4
     Partition 1: {'C4', 'C7', 'C8', 'C6'}
     Partition 2: {'C9', 'C3', 'C5', 'C10'}
        Level 2:
        Complete Set: {'C4', 'C7', 'C8', 'C6'}
        Start Partitions Cost: 2
         Partition 1: {'C4', 'C8'}
         Partition 2: {'C7', 'C6'}
        Final Partitions Cost: 0
         Partition 1: {'C7', 'C8'}
        

# Weighted Kernighan-Lin Partition

## Graph

In [ ]:
class CoreGraph:
    def __init__(self, cores, edges):
        self.cores = cores
        self.edges = edges
        self.weights = {edge: random.randint(1, 10) for edge in edges}

## Calculate Cost

In [ ]:
def calculate_cost(graph, partition1, partition2):
    cost = 0
    for edge, weight in graph.weights.items():
        if (edge[0] in partition1 and edge[1] in partition2) or (edge[0] in partition2 and edge[1] in partition1):
            cost += weight
    return cost

## Swap Core

In [ ]:
def get_next_move(graph, partition1, partition2):
    best_cost = calculate_cost( graph, partition1, partition2 )
    best_move = None
    for ci in partition1:
        for cj in partition2:
            current_cost = calculate_cost(graph, partition1 - {ci} | {cj}, partition2 - {cj} | {ci})
            if current_cost < best_cost:
                best_cost = current_cost
                best_move = (ci, cj)
    return best_move

## Kernighan_Lin Partition

In [ ]:
def kernighan_lin(graph, partition1, partition2, level=0):
    best_partition1, best_partition2 = partition1.copy(), partition2.copy()
    current_partition1, current_partition2 = partition1.copy(), partition2.copy()

    max_iterations = 50

    for iteration in range(max_iterations):
        next_move = get_next_move(graph, current_partition1, current_partition2)
        if next_move is None:
            break  # Salimos del ciclo si no encontramos una mejor jugada
        ci, cj = next_move
        current_partition1.remove(ci)
        current_partition2.remove(cj)
        current_partition1.add(cj)
        current_partition2.add(ci)

        if calculate_cost(graph, current_partition1, current_partition2) < calculate_cost(graph, best_partition1, best_partition2):
            best_partition1, best_partition2 = current_partition1.copy(), current_partition2.copy()

    if len(best_partition1) == len(partition1) and len(best_partition2) == len(partition2):
        return best_partition1, best_partition2

    p1, p2 = best_partition1, best_partition2
    return p1, p2

## Extend Kernighan-Lin Partition

In [ ]:
def kl_partitioning(graph, level, cores_to_be_partitioned):
  cores_to_be_partitioned = set(cores_to_be_partitioned)  # Convert to set to allow set operations

  if len(cores_to_be_partitioned) < 4:
    return cores_to_be_partitioned

  cores_list = list(cores_to_be_partitioned)

  partition1 = set(random.sample(cores_list, len(cores_list) // 2))
  partition2 = cores_to_be_partitioned - partition1

  indent = " " * (level * 4)
  print(indent + f"Level {level}:")
  print(indent + f"Complete Set: {cores_to_be_partitioned}")
  print(indent + f"Start Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1, partition2 = kernighan_lin(graph, partition1, partition2)

  indent = " " * (level * 4)
  print(indent + f"Final Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1_1, partition1_2 = kl_partitioning(graph, level + 1, partition1)
  partition2_1, partition2_2 = kl_partitioning(graph, level + 1, partition2)

  partition1 = (partition1_1, partition1_2)
  partition2 = (partition2_1, partition2_2)

  return partition1, partition2


## Test

In [ ]:
# Ejemplo de uso
cores = {'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16'}
edges = {('C1', 'C2'), ('C1', 'C3'), ('C2', 'C4'), ('C3', 'C4'), ('C3', 'C5'), ('C4', 'C6'), ('C5', 'C6'),
         ('C7', 'C8'), ('C7', 'C9'), ('C8', 'C10'), ('C9', 'C10'), ('C9', 'C11'), ('C10', 'C12'), ('C11', 'C12'),
         ('C13', 'C14'), ('C13', 'C15'), ('C14', 'C16'), ('C15', 'C16')}
graph = CoreGraph(cores, edges)

partition1, partition2 = kl_partitioning(graph, 0, cores)

print( "Cost:" f"{calculate_cost( graph, partition1, partition2 )}" )
print("Final Partition 1:", partition1)
print("Final Partition 2:", partition2)

Level 0:
Complete Set: {'C15', 'C4', 'C7', 'C10', 'C2', 'C1', 'C13', 'C16', 'C6', 'C14', 'C9', 'C8', 'C3', 'C11', 'C5', 'C12'}
Start Partitions Cost: 54
 Partition 1: {'C15', 'C4', 'C6', 'C11', 'C8', 'C3', 'C1', 'C13'}
 Partition 2: {'C7', 'C16', 'C14', 'C9', 'C10', 'C2', 'C5', 'C12'}
Final Partitions Cost: 5
 Partition 1: {'C13', 'C4', 'C6', 'C14', 'C3', 'C2', 'C1', 'C5'}
 Partition 2: {'C15', 'C7', 'C16', 'C9', 'C11', 'C8', 'C10', 'C12'}
    Level 1:
    Complete Set: {'C13', 'C4', 'C6', 'C14', 'C3', 'C2', 'C1', 'C5'}
    Start Partitions Cost: 29
     Partition 1: {'C2', 'C5', 'C3', 'C14'}
     Partition 2: {'C4', 'C1', 'C6', 'C13'}
    Final Partitions Cost: 13
     Partition 1: {'C2', 'C1', 'C3', 'C5'}
     Partition 2: {'C13', 'C4', 'C6', 'C14'}
        Level 2:
        Complete Set: {'C2', 'C1', 'C3', 'C5'}
        Start Partitions Cost: 6
         Partition 1: {'C3', 'C5'}
         Partition 2: {'C2', 'C1'}
        Final Partitions Cost: 6
         Partition 1: {'C3', 'C5'}
   

# Weighted Internal-External Kernighan-Lin Partition

## Graph

In [ ]:
import random
class CoreGraph:
    def __init__(self, cores, edges):
        self.cores = cores
        self.edges = edges
        self.weights = {edge: random.randint(1, 10) for edge in edges}

## Calculate Cost

In [ ]:
def calculate_cost(graph, partition1, partition2):
    external_cost = 0
    internal_cost = 0

    for edge, weight in graph.weights.items():
        if (edge[0] in partition1 and edge[1] in partition2) or (edge[0] in partition2 and edge[1] in partition1):
            external_cost += weight
        elif (edge[0] in partition1 and edge[1] in partition1) or (edge[0] in partition2 and edge[1] in partition2):
            internal_cost += weight

    return external_cost - internal_cost

## Swap Core

In [ ]:
def get_next_move(graph, partition1, partition2, locked_vertices):
  best_cost = calculate_cost( graph, partition1, partition2 )
  best_move = None
  for ci in partition1:
    if ci in locked_vertices:
      continue
    for cj in partition2:
      if cj in locked_vertices:
        continue
      current_cost = calculate_cost(graph, partition1 - {ci} | {cj}, partition2 - {cj} | {ci})
      if current_cost < best_cost:
        best_cost = current_cost
        best_move = (ci, cj)
  return best_move

## Kernighan_Lin Partition

In [ ]:
def kernighan_lin(graph, partition1, partition2, level=0):
    best_partition1, best_partition2 = partition1.copy(), partition2.copy()
    current_partition1, current_partition2 = partition1.copy(), partition2.copy()

    max_iterations = 5

    for iteration in range(max_iterations):
      locked_vertices = set()
      print( iteration )
      while( len(current_partition1) > 1 ):
        next_move = get_next_move(graph, current_partition1, current_partition2, locked_vertices)
        print( next_move )
        if next_move is None:
            break  # Salimos del ciclo si no encontramos una mejor jugada
        ci, cj = next_move
        current_partition1.remove(ci)
        current_partition2.remove(cj)
        current_partition1.add(cj)
        current_partition2.add(ci)
        locked_vertices.add(ci)
        locked_vertices.add(cj)

        if calculate_cost(graph, current_partition1, current_partition2) < calculate_cost(graph, best_partition1, best_partition2):
            best_partition1, best_partition2 = current_partition1.copy(), current_partition2.copy()

    if len(best_partition1) == len(partition1) and len(best_partition2) == len(partition2):
      return best_partition1, best_partition2

    p1, p2 = best_partition1, best_partition2
    return p1, p2

## Extend Kernighan-Lin Partition

In [ ]:
def kl_partitioning(graph, level, cores_to_be_partitioned):
  cores_to_be_partitioned = set(cores_to_be_partitioned)  # Convert to set to allow set operations

  if len(cores_to_be_partitioned) < 4:
    return cores_to_be_partitioned

  cores_list = list(cores_to_be_partitioned)

  partition1 = set(random.sample(cores_list, len(cores_list) // 2))
  partition2 = cores_to_be_partitioned - partition1

  indent = " " * (level * 4)
  print(indent + f"Level {level}:")
  print(indent + f"Complete Set: {cores_to_be_partitioned}")
  print(indent + f"Start Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1, partition2 = kernighan_lin(graph, partition1, partition2)

  indent = " " * (level * 4)
  print(indent + f"Final Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1_1, partition1_2 = kl_partitioning(graph, level + 1, partition1)
  partition2_1, partition2_2 = kl_partitioning(graph, level + 1, partition2)

  partition1 = (partition1_1, partition1_2)
  partition2 = (partition2_1, partition2_2)

  return partition1, partition2


## Test

In [ ]:
# Ejemplo de uso
cores = {'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16'}
edges = {('C1', 'C2'), ('C1', 'C3'), ('C2', 'C4'), ('C3', 'C4'), ('C3', 'C5'), ('C4', 'C6'), ('C5', 'C6'),
         ('C7', 'C8'), ('C7', 'C9'), ('C8', 'C10'), ('C9', 'C10'), ('C9', 'C11'), ('C10', 'C12'), ('C11', 'C12'),
         ('C13', 'C14'), ('C13', 'C15'), ('C14', 'C16'), ('C15', 'C16')}
graph = CoreGraph(cores, edges)

partition1, partition2 = kl_partitioning(graph, 0, cores)

print( "Cost:" f"{calculate_cost( graph, partition1, partition2 )}" )
print("Final Partition 1:", partition1)
print("Final Partition 2:", partition2)

Level 0:
Complete Set: {'C5', 'C7', 'C2', 'C14', 'C6', 'C4', 'C1', 'C3', 'C9', 'C10', 'C12', 'C11', 'C15', 'C13', 'C16', 'C8'}
Start Partitions Cost: 49
 Partition 1: {'C5', 'C7', 'C12', 'C11', 'C15', 'C2', 'C14', 'C6'}
 Partition 2: {'C4', 'C1', 'C3', 'C9', 'C10', 'C13', 'C16', 'C8'}
0
('C2', 'C13')
('C7', 'C16')
('C5', 'C9')
('C6', 'C10')
None
1
None
2
None
3
None
4
None
Final Partitions Cost: -95
 Partition 1: {'C9', 'C10', 'C12', 'C11', 'C15', 'C13', 'C14', 'C16'}
 Partition 2: {'C4', 'C1', 'C3', 'C5', 'C7', 'C2', 'C6', 'C8'}
    Level 1:
    Complete Set: {'C9', 'C10', 'C12', 'C11', 'C15', 'C13', 'C14', 'C16'}
    Start Partitions Cost: -7
     Partition 1: {'C9', 'C12', 'C15', 'C11'}
     Partition 2: {'C16', 'C14', 'C10', 'C13'}
0
('C15', 'C10')
None
1
None
2
None
3
None
4
None
    Final Partitions Cost: -55
     Partition 1: {'C9', 'C12', 'C11', 'C10'}
     Partition 2: {'C14', 'C15', 'C16', 'C13'}
        Level 2:
        Complete Set: {'C9', 'C12', 'C11', 'C10'}
        Start

# Weighted Example - Algorithm Kernighan-Lin

## Funciones

In [ ]:
import numpy as np

def create_external_adj_matrix(partition1, partition2, num_nodes):
  external_adj_matrix = np.zeros((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      external_adj_matrix[node1][node2] = 1
      external_adj_matrix[node2][node1] = 1

  return external_adj_matrix


def create_internal_adj_matrix( partition1, partition2, num_nodes ):

  internal_adj_matrix = np.ones((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      internal_adj_matrix[node1][node2] = 0
      internal_adj_matrix[node2][node1] = 0

  return internal_adj_matrix

def create_external_cost_matrix( cost_matrix, num_nodes ):

  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
  external_cost_matrix = np.zeros((num_nodes,num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
      external_cost_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

  return external_cost_matrix


def create_internal_cost_matrix( cost_matrix, num_nodes ):

  internal_adj_matrix = create_internal_adj_matrix(partition1, partition2, num_nodes)
  internal_cost_matrix = np.zeros((num_nodes,num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
      if i != j:
        internal_cost_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

  return internal_cost_matrix


def calculate_difference_cost_vector(  partition1, partition2, num_nodes ):

  external_cost_matrix = create_external_cost_matrix( cost_matrix, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, num_nodes )
  difference_cost_matrix = np.zeros((num_nodes, num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
     difference_cost_matrix[i][j] += external_cost_matrix[i][j] - internal_cost_matrix[i][j]


  print("*-"*20)
  print( "External Cost Matrix" )
  print( external_cost_matrix )


  print("-"*40)
  print( "Internal Cost Matrix" )
  print( internal_cost_matrix )

  print("*-"*20)
  print( "\n" )

  return np.dot([1,1,1,1,1,1],difference_cost_matrix)

def update_difference_cost_vector(  difference_cost_vector, partition1, partition2, available_nodes, swap_nodes  ):

    internal_adj_matrix = create_internal_adj_matrix( partition1, partition2, num_nodes)
    difference_cost_internal_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_internal_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

    external_adj_matrix = create_external_adj_matrix( partition1, partition2, num_nodes)
    difference_cost_external_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
          difference_cost_external_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

    difference_cost_prime_matrix = np.zeros((num_nodes,num_nodes))
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_prime_matrix[i][j] = difference_cost_internal_matrix[i][j] - difference_cost_external_matrix[i][j]

    difference_cost_vector          = np.multiply(available_nodes, difference_cost_vector)
    difference_cost_internal_vector = np.dot(swap_nodes, difference_cost_internal_matrix)
    difference_cost_external_vector = np.dot(swap_nodes, difference_cost_external_matrix)
    difference_cost_prime_vector    = difference_cost_vector + 2 * difference_cost_internal_vector - 2 * difference_cost_external_vector

    print(internal_adj_matrix)
    print("*-"*20)
    print( "Difference Cost Internal Matrix")
    for row in difference_cost_internal_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost Internal Vector")
    print( difference_cost_internal_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost External Matrix")
    for row in difference_cost_external_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost External Vector")
    print( difference_cost_external_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost Vector")
    print( difference_cost_vector)

    print("-"*40)

    print( "Difference Cost Updated Vector")
    print( difference_cost_prime_vector )

    print("*-"*20)
    print("\n")

    return difference_cost_prime_vector

def calculate_gain_cost_matrix( difference_cost_vector, external_adj_matrix, cost_matrix ):
  gain_cost_matrix = np.full((num_nodes,num_nodes), 0)

  for i in range(num_nodes):
    for j in range(num_nodes):
      if external_adj_matrix[i][j] == 1:
        gain_cost_matrix[i][j] = difference_cost_vector[i] + difference_cost_vector[j] - 2 * cost_matrix[i][j]

  return gain_cost_matrix


def swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes ):
  partition1_copy = partition1.copy()
  partition2_copy = partition2.copy()
  max_gain = -float('inf')
  max_i, max_j = None, None

  for i in range( num_nodes ):
    for j in range( num_nodes ):
      if max_gain < gain_cost_matrix[i][j] and gain_cost_matrix[i][j] != 0 and available_nodes[i] == 1 and available_nodes[j] == 1:
        max_gain = gain_cost_matrix[i][j]
        max_i = i
        max_j = j

  print( max_i, max_j, max_gain )
  next_move = [max_gain, max_i, max_j]
  for ci in partition1:
    for cj in partition2:
      if ci == max_i and cj == max_j:
        partition1_copy.remove(ci)
        partition2_copy.remove(cj)
        partition1_copy.add(cj)
        partition2_copy.add(ci)

  return partition1_copy, partition2_copy, next_move

def max_subarray_sum(next_moves):
    select_next_moves = []

    current_sum = 0
    max_sum = 0

    for next_move in next_moves:
        current_sum += int(next_move[0])
        if current_sum > max_sum:
          max_sum = current_sum
          select_next_moves.append( next_move )

    return max_sum, select_next_moves

def swap_cores(  partial_next_moves, partition1, partition2  ):
  for next_move in partial_next_moves:
    for ci in partition1:
     for cj in partition2:
        if ci == next_move[1] and cj == next_move[2]:
          partition1.remove(ci)
          partition2.remove(cj)
          partition1.add(cj)
          partition2.add(ci)
  return partition1, partition2


## Ejemplo

In [ ]:
# Ejemplo de uso
if __name__ == "__main__":
  partition1 = {0, 1, 2}
  partition2 = {3, 4, 5}
  num_nodes = 6
  next_moves = []

  cost_matrix = [ [0,1,2,3,2,4],
                  [1,0,1,4,2,1],
                  [2,1,0,3,2,1],
                  [3,4,3,0,4,3],
                  [2,2,2,4,0,2],
                  [4,1,1,3,2,0] ]




In [ ]:
  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
  internal_adj_matrix = create_internal_adj_matrix(partition1, partition2, num_nodes)
  #external_cost_matrix = create_external_cost_matrix( cost_matrix, external_adj_matrix, num_nodes )
  #internal_cost_matrix = create_internal_cost_matrix( cost_matrix, internal_adj_matrix, num_nodes )

  external_cost_matrix = create_external_cost_matrix( cost_matrix, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, num_nodes )

### Iteracion #1

#### Intercambio #1

In [ ]:
  difference_cost_vector =  calculate_difference_cost_vector( partition1, partition2, num_nodes  )
  gain_cost_matrix       =  calculate_gain_cost_matrix( difference_cost_vector, external_adj_matrix, cost_matrix )

  available_nodes = [1,1,1,1,1,1]
  partition1_swapped, partition2_swapped, next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes )

  next_moves.append( next_move )

  print("-"*40)
  print("Difference cost Vector")
  print( difference_cost_vector )
  print("-"*40)

  print("\n")

  print("-"*40)
  print("Gain Cost Matrix")
  print( gain_cost_matrix )
  print("-"*40)

  print("\n")

  print("-"*40)
  print("Particiones")
  print( "Partition1:", partition1_swapped )
  print( "Partition2:", partition2_swapped )

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
External Cost Matrix
[[0. 0. 0. 3. 2. 4.]
 [0. 0. 0. 4. 2. 1.]
 [0. 0. 0. 3. 2. 1.]
 [3. 4. 3. 0. 0. 0.]
 [2. 2. 2. 0. 0. 0.]
 [4. 1. 1. 0. 0. 0.]]
----------------------------------------
Internal Cost Matrix
[[0. 1. 2. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [2. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 4. 3.]
 [0. 0. 0. 4. 0. 2.]
 [0. 0. 0. 3. 2. 0.]]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


1 5 4
----------------------------------------
Difference cost Vector
[6. 5. 3. 3. 0. 1.]
----------------------------------------


----------------------------------------
Gain Cost Matrix
[[ 0  0  0  3  2 -1]
 [ 0  0  0  0  1  4]
 [ 0  0  0  0 -1  2]
 [ 3  0  0  0  0  0]
 [ 2  1 -1  0  0  0]
 [-1  4  2  0  0  0]]
----------------------------------------


----------------------------------------
Particiones
Partition1: {0, 2, 5}
Partition2: {1, 3, 4}


#### Intercambio #2

In [ ]:
  available_nodes = [1,0,1,1,1,0]
  swap_nodes      = [0,1,0,0,0,1]

  difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_vector, partition1, partition2, available_nodes, swap_nodes )
  gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1_swapped, partition2_swapped, num_nodes ), cost_matrix  )

  print(  gain_cost_matrix  )

  partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
  next_moves.append( next_move )

  print( "Partition1:", partition1_swapped )
  print( "Partition2:", partition2_swapped )

[[1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1.]]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Internal Matrix
[0. 1. 2. 0. 0. 0.] 3.0
[1. 0. 1. 0. 0. 0.] 2.0
[2. 1. 0. 0. 0. 0.] 3.0
[0. 0. 0. 0. 4. 3.] 7.0
[0. 0. 0. 4. 0. 2.] 6.0
[0. 0. 0. 3. 2. 0.] 5.0
----------------------------------------
Difference Cost Internal Vector
[1. 0. 1. 3. 2. 0.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost External Matrix
[0. 0. 0. 3. 2. 4.] 9.0
[0. 0. 0. 4. 2. 1.] 7.0
[0. 0. 0. 3. 2. 1.] 6.0
[3. 4. 3. 0. 0. 0.] 10.0
[2. 2. 2. 0. 0. 0.] 6.0
[4. 1. 1. 0. 0. 0.] 6.0
----------------------------------------
Difference Cost External Vector
[4. 1. 1. 4. 2. 1.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Vector
[6. 0. 3. 3. 0. 0.]
----------------------------------------
Difference Cost Updated Vector
[ 0. -2.  

#### Intercambio #3

In [ ]:
  available_nodes = [1,0,0,1,0,0]
  swap_nodes = [0,0,1,0,1,0]

  difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_prime_vector, partition1, partition2, available_nodes, swap_nodes )
  gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1, partition2, num_nodes ), cost_matrix  )
  print( gain_cost_matrix  )
  available_nodes = [1,0,0,1,0,0]
  partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
  next_moves.append( next_move )

[[1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1.]]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Internal Matrix
[0. 1. 2. 0. 0. 0.] 3.0
[1. 0. 1. 0. 0. 0.] 2.0
[2. 1. 0. 0. 0. 0.] 3.0
[0. 0. 0. 0. 4. 3.] 7.0
[0. 0. 0. 4. 0. 2.] 6.0
[0. 0. 0. 3. 2. 0.] 5.0
----------------------------------------
Difference Cost Internal Vector
[2. 1. 0. 4. 0. 2.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost External Matrix
[0. 0. 0. 3. 2. 4.] 9.0
[0. 0. 0. 4. 2. 1.] 7.0
[0. 0. 0. 3. 2. 1.] 6.0
[3. 4. 3. 0. 0. 0.] 10.0
[2. 2. 2. 0. 0. 0.] 6.0
[4. 1. 1. 0. 0. 0.] 6.0
----------------------------------------
Difference Cost External Vector
[2. 2. 2. 3. 2. 1.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Vector
[ 0. -0.  0.  1.  0. -0.]
----------------------------------------
Difference Cost Updated Vector
[ 0.

#### Movimientos Parciales

In [ ]:
  max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )
  print(max_partial_sum, partial_next_moves)

4 [[4, 1, 5]]


#### Intercambio

In [ ]:
  partition1, partition2 = swap_cores( partial_next_moves, partition1, partition2 )
  print( partition1 )
  print( partition2 )

{0, 2, 5}
{1, 3, 4}


### Iteracion #2

In [ ]:
  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
  internal_adj_matrix = create_internal_adj_matrix(partition1, partition2, num_nodes)
  #external_cost_matrix = create_external_cost_matrix( cost_matrix, external_adj_matrix, num_nodes )
  #internal_cost_matrix = create_internal_cost_matrix( cost_matrix, internal_adj_matrix, num_nodes )

  external_cost_matrix = create_external_cost_matrix( cost_matrix, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, num_nodes )
  next_moves = []

#### Intercambio #1

In [ ]:
  difference_cost_vector =  calculate_difference_cost_vector( partition1, partition2, num_nodes  )
  gain_cost_matrix       =  calculate_gain_cost_matrix( difference_cost_vector, external_adj_matrix, cost_matrix )

  available_nodes = [1,1,1,1,1,1]
  partition1_swapped, partition2_swapped, next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes )

  next_moves.append( next_move )

  print("-"*40)
  print("Difference cost Vector")
  print( difference_cost_vector )
  print("-"*40)

  print("\n")

  print("-"*40)
  print("Gain Cost Matrix")
  print( gain_cost_matrix )
  print("-"*40)

  print("\n")

  print("-"*40)
  print("Particiones")
  print( "Partition1:", partition1_swapped )
  print( "Partition2:", partition2_swapped )

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
External Cost Matrix
[[0. 1. 0. 3. 2. 0.]
 [1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 3. 2. 0.]
 [3. 0. 3. 0. 0. 3.]
 [2. 0. 2. 0. 0. 2.]
 [0. 1. 0. 3. 2. 0.]]
----------------------------------------
Internal Cost Matrix
[[0. 0. 2. 0. 0. 4.]
 [0. 0. 0. 4. 2. 0.]
 [2. 0. 0. 0. 0. 1.]
 [0. 4. 0. 0. 4. 0.]
 [0. 2. 0. 4. 0. 0.]
 [4. 0. 1. 0. 0. 0.]]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


2 4 -1
----------------------------------------
Difference cost Vector
[ 0. -3.  3.  1.  0.  1.]
----------------------------------------


----------------------------------------
Gain Cost Matrix
[[ 0 -5  0 -5 -4  0]
 [-5  0 -2  0  0 -4]
 [ 0 -2  0 -2 -1  0]
 [-5  0 -2  0  0 -4]
 [-4  0 -1  0  0 -3]
 [ 0 -4  0 -4 -3  0]]
----------------------------------------


----------------------------------------
Particiones
Partition1: {0, 4, 5}
Partition2: {1, 2, 3}


#### Intercambio #2

In [ ]:
  available_nodes = [1,1,0,1,0,1]
  swap_nodes      = [0,0,1,0,1,0]

  difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_vector, partition1, partition2, available_nodes, swap_nodes )
  gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1_swapped, partition2_swapped, num_nodes ), cost_matrix  )

  print(  gain_cost_matrix  )

  partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
  next_moves.append( next_move )

  print( "Partition1:", partition1_swapped )
  print( "Partition2:", partition2_swapped )

[[1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1. 0.]
 [1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1. 0.]
 [0. 1. 0. 1. 1. 0.]
 [1. 0. 1. 0. 0. 1.]]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Internal Matrix
[0. 0. 2. 0. 0. 4.] 6.0
[0. 0. 0. 4. 2. 0.] 6.0
[2. 0. 0. 0. 0. 1.] 3.0
[0. 4. 0. 0. 4. 0.] 8.0
[0. 2. 0. 4. 0. 0.] 6.0
[4. 0. 1. 0. 0. 0.] 5.0
----------------------------------------
Difference Cost Internal Vector
[2. 2. 0. 4. 0. 1.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost External Matrix
[0. 1. 0. 3. 2. 0.] 6.0
[1. 0. 1. 0. 0. 1.] 3.0
[0. 1. 0. 3. 2. 0.] 6.0
[3. 0. 3. 0. 0. 3.] 9.0
[2. 0. 2. 0. 0. 2.] 6.0
[0. 1. 0. 3. 2. 0.] 6.0
----------------------------------------
Difference Cost External Vector
[2. 1. 2. 3. 2. 2.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Vector
[ 0. -3.  0.  1.  0.  1.]
----------------------------------------
Difference Cost Updated Vector
[ 0. 

#### Intercambio #3

In [ ]:
  available_nodes = [0,0,0,1,0,1]
  swap_nodes      = [1,1,0,0,0,0]

  difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_prime_vector, partition1, partition2, available_nodes, swap_nodes )
  gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1, partition2, num_nodes ), cost_matrix  )
  print( gain_cost_matrix  )

  partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
  next_moves.append( next_move )

[[1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1. 0.]
 [1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1. 0.]
 [0. 1. 0. 1. 1. 0.]
 [1. 0. 1. 0. 0. 1.]]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Internal Matrix
[0. 0. 2. 0. 0. 4.] 6.0
[0. 0. 0. 4. 2. 0.] 6.0
[2. 0. 0. 0. 0. 1.] 3.0
[0. 4. 0. 0. 4. 0.] 8.0
[0. 2. 0. 4. 0. 0.] 6.0
[4. 0. 1. 0. 0. 0.] 5.0
----------------------------------------
Difference Cost Internal Vector
[0. 0. 2. 4. 2. 4.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost External Matrix
[0. 1. 0. 3. 2. 0.] 6.0
[1. 0. 1. 0. 0. 1.] 3.0
[0. 1. 0. 3. 2. 0.] 6.0
[3. 0. 3. 0. 0. 3.] 9.0
[2. 0. 2. 0. 0. 2.] 6.0
[0. 1. 0. 3. 2. 0.] 6.0
----------------------------------------
Difference Cost External Vector
[1. 1. 1. 3. 2. 1.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Vector
[ 0. -0. -0.  3. -0. -1.]
----------------------------------------
Difference Cost Updated Vector
[-2. 

#### Movimientos Parciales

In [ ]:
  max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )

  print(max_partial_sum, partial_next_moves)
  print( partial_next_moves  )

0 []
[]


#### Intercambio

In [ ]:
partition1, partition2 = swap_cores( partial_next_moves, partition1, partition2 )
print( partition1 )
print( partition2 )

{0, 2, 5}
{1, 3, 4}


# Implementación Algoritmo Kernighan-Lin

# Bibliotecas

In [ ]:
import numpy as np

## Funciones

In [ ]:
import numpy as np

def create_external_adj_matrix(partition1, partition2, num_nodes):
  external_adj_matrix = np.zeros((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      external_adj_matrix[node1][node2] = 1
      external_adj_matrix[node2][node1] = 1

  return external_adj_matrix


def create_internal_adj_matrix( partition1, partition2, num_nodes ):

  internal_adj_matrix = np.ones((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      internal_adj_matrix[node1][node2] = 0
      internal_adj_matrix[node2][node1] = 0

  return internal_adj_matrix

def create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes ):
  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
  external_cost_matrix = np.zeros((num_nodes,num_nodes))

  for i in range(num_nodes):
    for j in range(num_nodes):
      external_cost_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

  return external_cost_matrix


def create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes ):

  internal_adj_matrix = create_internal_adj_matrix(partition1, partition2, num_nodes)
  internal_cost_matrix = np.zeros((num_nodes,num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
      if i != j:
        internal_cost_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

  return internal_cost_matrix


def calculate_difference_cost_vector(  partition1, partition2, num_nodes ):

  external_cost_matrix = create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes )


  difference_cost_matrix = np.zeros((num_nodes, num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
     difference_cost_matrix[i][j] += external_cost_matrix[i][j] - internal_cost_matrix[i][j]

  '''
  print("*-"*20)
  print( "External Cost Matrix" )
  print( external_cost_matrix )


  print("-"*40)
  print( "Internal Cost Matrix" )
  print( internal_cost_matrix )

  print("*-"*20)
  print( "\n" )
  #'''
  #return np.dot([1,1,1,1,1,1],difference_cost_matrix)
  #return np.dot(np.array(num_nodes), difference_cost_matrix)
  return np.dot([1 for _ in range(num_nodes)],difference_cost_matrix)

def update_difference_cost_vector(  difference_cost_vector, partition1, partition2, available_nodes, swap_nodes  ):

    internal_adj_matrix = create_internal_adj_matrix( partition1, partition2, num_nodes)
    difference_cost_internal_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_internal_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

    external_adj_matrix = create_external_adj_matrix( partition1, partition2, num_nodes)
    difference_cost_external_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
          difference_cost_external_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

    difference_cost_prime_matrix = np.zeros((num_nodes,num_nodes))
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_prime_matrix[i][j] = difference_cost_internal_matrix[i][j] - difference_cost_external_matrix[i][j]

    difference_cost_vector          = np.multiply(available_nodes, difference_cost_vector)
    difference_cost_internal_vector = np.dot(swap_nodes, difference_cost_internal_matrix)
    difference_cost_external_vector = np.dot(swap_nodes, difference_cost_external_matrix)
    difference_cost_prime_vector    = difference_cost_vector + 2 * difference_cost_internal_vector - 2 * difference_cost_external_vector
    '''
    print(internal_adj_matrix)
    print("*-"*20)
    print( "Difference Cost Internal Matrix")
    for row in difference_cost_internal_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost Internal Vector")
    print( difference_cost_internal_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost External Matrix")
    for row in difference_cost_external_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost External Vector")
    print( difference_cost_external_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost Vector")
    print( difference_cost_vector)

    print("-"*40)

    print( "Difference Cost Updated Vector")
    print( difference_cost_prime_vector )

    print("*-"*20)
    print("\n")
    #'''
    return difference_cost_prime_vector

def calculate_gain_cost_matrix( difference_cost_vector, partition1, partition2 ,external_adj_matrix, cost_matrix ):
  gain_cost_matrix = np.full((num_nodes,num_nodes), 0)


  external_cost_matrix = create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes )

  for i in range(num_nodes):
    for j in range(num_nodes):
      if external_adj_matrix[i][j] == 1:
        gain_cost_matrix[i][j] = difference_cost_vector[i] + difference_cost_vector[j] - 2 * cost_matrix[i][j]

  return gain_cost_matrix


def swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes ):
  partition1_copy = set(partition1)
  partition2_copy = set(partition2)
  max_gain = -float('inf')
  max_i, max_j = None, None

  for i in range( num_nodes ):
    for j in range( num_nodes ):
      if max_gain < gain_cost_matrix[i][j] and gain_cost_matrix[i][j] != 0 and available_nodes[i] == 1 and available_nodes[j] == 1 and i in partition1_copy and j in partition2_copy:
        max_gain = gain_cost_matrix[i][j]
        max_i = i
        max_j = j

  print( max_i, max_j, max_gain )
  next_move = [max_gain, max_i, max_j]

  for ci in partition1_copy:
    for cj in partition2_copy:
      if max_i== ci and max_j == cj:
        partition1_copy.remove(max_i)
        partition2_copy.remove(max_j)
        partition1_copy.add(max_j)
        partition2_copy.add(max_i)
        print("Hubo intercambio Parcial!!!!!", (max_i, max_j))

  return partition1_copy, partition2_copy, next_move

def max_subarray_sum(next_moves):
    select_next_moves = []
    current_sum = 0
    max_sum = -np.inf

    for next_move in next_moves:
        current_sum += int(next_move[0])
        print( next_move, current_sum )
        if current_sum > max_sum:
          max_sum = current_sum
          select_next_moves.append( next_move )

    return max_sum, select_next_moves

def swap_cores(  partial_next_moves, partition1, partition2  ):
  for next_move in partial_next_moves:
    for ci in partition1:
     for cj in partition2:
        if ci == next_move[1] and cj == next_move[2]:
          partition1.remove(ci)
          partition2.remove(cj)
          partition1.add(cj)
          partition2.add(ci)
  return partition1, partition2

def swap_cores(partial_next_moves, partition1, partition2):
    new_partition1 = set(partition1)
    new_partition2 = set(partition2)

    for next_move in partial_next_moves:
        node_from = next_move[1]
        node_to = next_move[2]

        if node_from in new_partition1 and node_to in new_partition2:
            new_partition1.remove(node_from)
            new_partition2.remove(node_to)
            new_partition1.add(node_to)
            new_partition2.add(node_from)
            print( "Hubo intercambio Final!!!!!" + f"{node_from,node_to}"  )

    return new_partition1, new_partition2

def update_available_nodes( available_nodes, swap_nodes ):
  return  [a ^ b for a, b in zip(available_nodes, swap_nodes)]

def update_swap_nodes( next_move ):
  indices = next_move[1:3]
  swap_nodes = [1 if i in indices else 0 for i in range(num_nodes)]
  return swap_nodes


## Ejemplo

# Iteración # 1

In [ ]:
  def kernighan_lin( partition1, partition2, cost_matrix ):
    next_moves = []

    number_of_iteration   = 0
    number_of_interchange = 0

    if number_of_iteration == 0:
      print( "### Iteración 0 ###"  )
      number_of_interchange = 0
      print( "  # Intercambio 0 #" )

      if number_of_interchange == 0:
        difference_cost_vector =  calculate_difference_cost_vector( partition1, partition2, num_nodes  )
        gain_cost_matrix       =  calculate_gain_cost_matrix( difference_cost_vector, external_adj_matrix, cost_matrix )

        available_nodes = [1,1,1,1,1,1]
        partition1_swapped, partition2_swapped, next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes )

        next_moves.append( next_move )

        print("-"*40)
        print("Difference cost Vector")
        print( difference_cost_vector )
        print("-"*40)

        print("\n")

        print("-"*40)
        print("Gain Cost Matrix")
        print( gain_cost_matrix )
        print("-"*40)

        print("\n")

        print("-"*40)
        print("Particiones")
        print( "Partition1:", partition1_swapped )
        print( "Partition2:", partition2_swapped )

        number_of_interchange = 1

      if number_of_interchange == 1:
        print( "  # Intercambio 1 #" )
        #available_nodes = [1,0,1,1,1,0]
        swap_nodes      = [0,1,0,0,0,1]


        swap_nodes      = update_swap_nodes( next_move )
        available_nodes = update_available_nodes( available_nodes, swap_nodes )

        difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_vector, partition1, partition2, available_nodes, swap_nodes )
        gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1_swapped, partition2_swapped, num_nodes ), cost_matrix  )

        print(  gain_cost_matrix  )

      partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
      next_moves.append( next_move )

      print( "Partition1:", partition1_swapped )
      print( "Partition2:", partition2_swapped )

      number_of_interchange = 2

      if number_of_interchange == 2:
        print( "  # Intercambio 2 #" )
        #available_nodes = [1,0,0,1,0,0]
        swap_nodes = [0,0,1,0,1,0]

        swap_nodes      = update_swap_nodes( next_move )
        available_nodes = update_available_nodes( available_nodes, swap_nodes )

        difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_prime_vector, partition1, partition2, available_nodes, swap_nodes )
        gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1, partition2, num_nodes ), cost_matrix  )
        print( gain_cost_matrix  )

        partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
        next_moves.append( next_move )

      max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )
      print(max_partial_sum, partial_next_moves)

      partition1, partition2 = swap_cores( partial_next_moves, partition1, partition2 )
      print( partition1 )
      print( partition2 )

      number_of_iteration = 1

    if number_of_iteration == 1:
      print( "### Iteración 1 ###"  )
      print( partition1 )
      print( partition2 )

      external_adj_matrix_1 = create_external_adj_matrix(partition1, partition2, num_nodes)
      internal_adj_matrix_1 = create_internal_adj_matrix(partition1, partition2, num_nodes)

      external_cost_matrix_1 = create_external_cost_matrix( cost_matrix, num_nodes )
      internal_cost_matrix_1 = create_internal_cost_matrix( cost_matrix, num_nodes )
      number_of_interchange = 0

      print( "# Intercambio 0 #" )
      if number_of_interchange == 0:

        difference_cost_vector =  calculate_difference_cost_vector( partition1, partition2, num_nodes  )
        gain_cost_matrix       =  calculate_gain_cost_matrix( difference_cost_vector, external_adj_matrix_1, cost_matrix )

        available_nodes = [1,1,1,1,1,1]
        partition1_swapped, partition2_swapped, next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes )

        next_moves.append( next_move )

        print("-"*40)
        print("Difference cost Vector")
        print( difference_cost_vector )
        print("-"*40)

        print("\n")

        print("-"*40)
        print("Gain Cost Matrix")
        print( gain_cost_matrix )
        print("-"*40)

        print("\n")

        print("-"*40)
        print("Particiones")
        print( "Partition1:", partition1_swapped )
        print( "Partition2:", partition2_swapped )

        number_of_interchange = 1

      if number_of_interchange == 1:
        print( "# Intercambio 1 #" )

        #available_nodes = [1,1,0,1,0,1]
        swap_nodes      = [0,0,1,0,1,0]

        swap_nodes      = update_swap_nodes( next_move )
        available_nodes = update_available_nodes( available_nodes, swap_nodes )

        difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_vector, partition1, partition2, available_nodes, swap_nodes )
        gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1_swapped, partition2_swapped, num_nodes ), cost_matrix  )

        print(  gain_cost_matrix  )

        partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
        next_moves.append( next_move )

        print( "Partition1:", partition1_swapped )
        print( "Partition2:", partition2_swapped )
        number_of_interchange = 2

      if number_of_interchange == 2:
        print( "# Intercambio 2 #" )
        #available_nodes = [0,0,0,1,0,1]
        swap_nodes      = [1,1,0,0,0,0]


        swap_nodes      = update_swap_nodes( next_move )
        available_nodes = update_available_nodes( available_nodes, swap_nodes )

        print( next_move  )
        print( available_nodes )
        print( swap_nodes )

        difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_prime_vector, partition1, partition2, available_nodes, swap_nodes )
        gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1, partition2, num_nodes ), cost_matrix  )
        print( gain_cost_matrix  )

        partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
        next_moves.append( next_move )

    max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )
    print(max_partial_sum, partial_next_moves)
    print( partial_next_moves  )


    partition1, partition2 = swap_cores( partial_next_moves, partition1, partition2 )
    print( partition1 )
    print( partition2 )




# Iteracion #*2*

In [ ]:
def kernighan_lin( partition1, partition2, cost_matrix ):

  number_of_iteration   = 0
  number_of_interchange = 0

  for number_of_iteration in range(100):
    next_moves = []
    print( "\n"*2 )
    print( "### Iteración "+ f"{number_of_iteration}" + "###"  )
    print( "Particiones Iniciales" )
    print( partition1 )
    print( partition2 )

    available_nodes = [1 for _ in range(num_nodes)]

    external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
    #internal_adj_matrix = create_#(partition1, partition2, num_nodes)
    external_cost_matrix = create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
    #internal_cost_matrix = create_#internal_cost_matrix( cost_matrix, num_nodes )

    number_of_interchange = 0

    while np.sum(available_nodes) :

      if number_of_interchange == 0:
        print( "  # Intercambio #" + f"{number_of_interchange}" )
        print( "Particiones Iniciales" )
        print( partition1 )
        print( partition2 )
        difference_cost_vector =  calculate_difference_cost_vector( partition1, partition2, num_nodes  )
        gain_cost_matrix       =  calculate_gain_cost_matrix( difference_cost_vector, partition1, partition2, external_adj_matrix, cost_matrix )
        partition1_swapped, partition2_swapped, next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes )

      else:
        print( "  # Intercambio #" + f"{number_of_interchange}" )
        difference_cost_vector =  update_difference_cost_vector( difference_cost_vector, partition1, partition2, available_nodes, swap_nodes )
        gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_vector, partition1_swapped, partition2_swapped,  create_external_adj_matrix( partition1_swapped, partition2_swapped, num_nodes ), cost_matrix  )
        partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )

      next_moves.append( next_move )
      swap_nodes      = update_swap_nodes( next_move )
      available_nodes = update_available_nodes( available_nodes, swap_nodes )
      number_of_interchange += 1
      '''
      print("-"*40)
      print("Difference cost Vector")
      print( difference_cost_vector )
      print("-"*40)

      print("\n")

      print("-"*40)
      print("Gain Cost Matrix")
      print( gain_cost_matrix )
      print("-"*40)

      print("\n")

      print("-"*40)
      print("Particiones Potenciales")
      print( "Movimiento Potencial:", next_move )
      print( "Partition1_potencial:", partition1_swapped )
      print( "Partition2_potencial:", partition2_swapped )
      #'''


    max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )
    print(max_partial_sum, partial_next_moves)

    print( "Movimientos Potenciales: ", next_moves )
    print( "Movimientos Finales:", partial_next_moves )


    if max_partial_sum <= 0:
      print("Terminando debido a que max_partial_sum es igual o menor a 0")
      #break
      print( "Particiones Finales" )
      print( partition1 )
      print( partition2 )
      return partition1, partition2


    partition1_post, partition2_post = swap_cores( partial_next_moves, partition1, partition2 )

    print(  )

    if partition1_post == partition2 and partition2_post == partition1:
      print( "Particiones Finales Iteracion" )
      print( partition1, partition2_post )
      print( partition2, partition1_post )
      return partition1, partition2

    partition1 = partition1_post
    partition2 = partition2_post






# Ejemplo

In [ ]:
# Ejemplo de uso
if __name__ == "__main__":


  nodes = {0,1,2,3,4,5}

  partition1 = {0, 1, 2}
  partition2 = {3, 4, 5}

  num_nodes = 6

  cost_matrix = [ [0,1,2,3,2,4],
                  [1,0,1,4,2,1],
                  [2,1,0,3,2,1],
                  [3,4,3,0,4,3],
                  [2,2,2,4,0,2],
                  [4,1,1,3,2,0] ]




In [ ]:
#VOPD
nodes = {0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15}
num_nodes = len(nodes)

partition1 = {0,1,2,3,4,5,6,7}
partition2 = {8,9,10,11,12,13,14,15}

partition1 = {0,1,2,3}
partition2 = {4,5,6,7}


partition1 = {8,9,10,11}
partition2 = {12,13,14,15}

num_nodes = 8

#partition1 = {0,9,2,3,11,5,12,14}
#partition2 = {8,1,10,4,6,13,7,15}

cost_matrix = [
    [  0.,  70.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
    [ 70.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
    [  0., 362.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
    [  0.,   0., 362.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  49.],
    [  0.,   0.,   0., 362.,   0., 357.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
    [  0.,   0.,   0.,   0., 357.,   0., 353.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
    [  0.,   0.,   0.,   0.,   0., 353.,   0., 300.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0., 300.,   0., 313., 500.,   0.,   0.,   0.,   0.,   0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0., 313.,   0.,  94.,   0.,   0.,   0.,   0.,   0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 313.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  16.,   0.,   0.,   0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,  16.,   0.,   0.,  16.,   0.,   0.,   0.,  16.,   0.,   0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 157.,   0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  16.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  16.,   0.,  16.,   0.,   0.,   0.],
    [  0.,   0.,   0.,   0.,  49.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]
]

cost_matrix = [
    [  0.,  70.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,  49.],
    [  0.,   0.,   0.,   0.,   0., 357.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0., 353.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0., 300.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 313., 500.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  94.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 313.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  16.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,  16.,   0.,   0.,  16.,   0.,   0.,   0.,  16.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 157.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 16.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  16.,   0.,  16.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,  27.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.]
]



In [ ]:
nodes = {0,1,2,3,4,5,6,7}

partition1 = {0, 1, 2, 3}
partition2 = {4, 5, 6, 7}

num_nodes = 8

cost_matrix = [ [0,200,0,0,0,0,0,0],
                [200,0,10,0,10,0,0,0],
                [0,10,0,1000,100,0,0,0],
                [0,0,1000,0,0,0,0,0],
                [0,10,100,0,0,200,0,0],
                [0,0,0,0,200,0,10,0],
                [0,0,0,0,0,10,0,1000],
                [0,0,0,0,0,0,1000,0]]


In [ ]:
partition1_kl, partition2_kl = kernighan_lin( partition1, partition2, cost_matrix )

print(  "\n" )
print( "Particiones Iniciales" + "\t"*2 + "Particiones Finales" )
print(  f"{partition1}" + "\t"*1 + f"{partition1_kl}" )
print(  f"{partition2}" + "\t"*1 + f"{partition2_kl}" )

# Version #*3*

## Funciones

In [87]:
import numpy as np
import math

def create_external_adj_matrix(partition1, partition2, num_nodes):
  external_adj_matrix = np.zeros((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      external_adj_matrix[node1][node2] = 1
      external_adj_matrix[node2][node1] = 1

  return external_adj_matrix


def create_internal_adj_matrix( partition1, partition2, num_nodes ):

  internal_adj_matrix = np.ones((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      internal_adj_matrix[node1][node2] = 0
      internal_adj_matrix[node2][node1] = 0

  return internal_adj_matrix

def create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes ):
  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
  external_cost_matrix = np.zeros((num_nodes,num_nodes))

  for i in range(num_nodes):
    for j in range(num_nodes):
      external_cost_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

  return external_cost_matrix


def create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes ):

  internal_adj_matrix = create_internal_adj_matrix(partition1, partition2, num_nodes)
  internal_cost_matrix = np.zeros((num_nodes,num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
      if i != j:
        internal_cost_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

  return internal_cost_matrix


def calculate_difference_cost_vector( cost_matrix, partition1, partition2, num_nodes ):

  external_cost_matrix = create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes )


  difference_cost_matrix = np.zeros((num_nodes, num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
     difference_cost_matrix[i][j] += external_cost_matrix[j][i] - internal_cost_matrix[j][i]

  #
  '''
  print("*-"*20)
  print( "External Cost Matrix" )
  print( external_cost_matrix )


  print("-"*40)
  print( "Internal Cost Matrix" )
  print( internal_cost_matrix )

  print("*-"*20)
  print( "\n" )
  #'''
  #return np.dot([1,1,1,1,1,1],difference_cost_matrix)
  #return np.dot(np.array(num_nodes), difference_cost_matrix)
  return np.dot([1 for _ in range(num_nodes)], difference_cost_matrix)

def update_difference_cost_vector( cost_matrix, difference_cost_vector, partition1, partition2, available_nodes, swap_nodes  ):

    internal_adj_matrix = create_internal_adj_matrix( partition1, partition2, num_nodes)
    difference_cost_internal_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_internal_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

    external_adj_matrix = create_external_adj_matrix( partition1, partition2, num_nodes)
    difference_cost_external_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
          difference_cost_external_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

    difference_cost_prime_matrix = np.zeros((num_nodes,num_nodes))
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_prime_matrix[i][j] = difference_cost_internal_matrix[i][j] - difference_cost_external_matrix[i][j]

    difference_cost_vector          = np.multiply(available_nodes, difference_cost_vector)
    difference_cost_internal_vector = np.dot(swap_nodes, difference_cost_internal_matrix)
    difference_cost_external_vector = np.dot(swap_nodes, difference_cost_external_matrix)
    difference_cost_prime_vector    = difference_cost_vector + 2 * difference_cost_internal_vector - 2 * difference_cost_external_vector

    #'''
    print(internal_adj_matrix)
    print("*-"*20)
    print( "Difference Cost Internal Matrix")
    for row in difference_cost_internal_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost Internal Vector")
    print( difference_cost_internal_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost External Matrix")
    for row in difference_cost_external_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost External Vector")
    print( difference_cost_external_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost Vector")
    print( difference_cost_vector)

    print("-"*40)

    print( "Difference Cost Updated Vector")
    print( difference_cost_prime_vector )

    print("*-"*20)
    print("\n")
    #'''
    return difference_cost_prime_vector

def calculate_gain_cost_matrix( difference_cost_vector, partition1, partition2 ,external_adj_matrix, cost_matrix ):
  gain_cost_matrix = np.full((num_nodes,num_nodes), 0)

  external_cost_matrix = create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)

  for i in range(num_nodes):
    for j in range(num_nodes):
      if external_adj_matrix[i][j] == 1:
        gain_cost_matrix[i][j] = difference_cost_vector[i] + difference_cost_vector[j] - 2 * cost_matrix[i][j]

  return gain_cost_matrix


def swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes ):
  partition1_copy = set(partition1)
  partition2_copy = set(partition2)
  max_gain = float('inf')
  max_i, max_j = None, None

  for i in range( num_nodes ):
    for j in range( num_nodes ):
      if gain_cost_matrix[i][j] < max_gain  and gain_cost_matrix[i][j] != 0 and available_nodes[i] == 1 and available_nodes[j] == 1 and i in partition1_copy and j in partition2_copy:
        max_gain = gain_cost_matrix[i][j]
        max_i = i
        max_j = j

  #print( max_i, max_j, max_gain )
  next_move = [ max_i, max_j, max_gain]

  for ci in partition1_copy:
    for cj in partition2_copy:
      if max_i== ci and max_j == cj:
        partition1_copy.remove(max_i)
        partition2_copy.remove(max_j)
        partition1_copy.add(max_j)
        partition2_copy.add(max_i)
        #print("Hubo intercambio Parcial!!!!!", (max_i, max_j))

  return next_move

def max_subarray_sum(next_moves):
    select_next_moves = []
    current_sum = 0
    max_sum = np.inf

    for next_move in next_moves:
      if math.isfinite(next_move[2]):
        current_sum = int(next_move[2])
        #print( next_move, current_sum )
        if current_sum < max_sum:
          max_sum = current_sum
          select_next_moves.append( next_move )

    return max_sum, select_next_moves

def swap_cores(  partial_next_moves, partition1, partition2  ):
  for next_move in partial_next_moves:
    for ci in partition1:
     for cj in partition2:
        if ci == next_move[0] and cj == next_move[1]:
          partition1.remove(ci)
          partition2.remove(cj)
          partition1.add(cj)
          partition2.add(ci)
  return partition1, partition2

def swap_cores(partial_next_moves, partition1, partition2):
    new_partition1 = set(partition1)
    new_partition2 = set(partition2)

    for next_move in partial_next_moves:
        node_from = next_move[0]
        node_to = next_move[1]

        if node_from in new_partition1 and node_to in new_partition2:
            new_partition1.remove(node_from)
            new_partition2.remove(node_to)
            new_partition1.add(node_to)
            new_partition2.add(node_from)
            #print( "Hubo intercambio Final!!!!!" + f"{node_from,node_to}"  )

    return new_partition1, new_partition2

def update_available_nodes( available_nodes, swap_nodes, num_nodes ):
  return  [a ^ b for a, b in zip(available_nodes, swap_nodes)]

def update_swap_nodes( next_move ):
  indices = next_move[0:2]
  swap_nodes = [1 if i in indices else 0 for i in range(num_nodes)]
  return swap_nodes


## Topology routers

In [88]:
import numpy as np

def create_hop_matrix_routers( adj_matrix_routers ):
  hop_matrix_routers = adj_matrix_routers.astype(float)
  hop_matrix_routers[hop_matrix_routers == 0] = np.inf

  for intermediate_router in range( adj_matrix_routers.shape[1] ):
    for source_router in range( adj_matrix_routers.shape[0] ):
      for destination_router in range( adj_matrix_routers.shape[1] ):
        if source_router == destination_router:
          hop_matrix_routers[source_router, destination_router] = 0
        else:
          hop_matrix_routers[source_router, destination_router]  = min(
              hop_matrix_routers[source_router, destination_router],
              hop_matrix_routers[source_router, intermediate_router] + hop_matrix_routers[intermediate_router, destination_router]
          )
  hop_matrix_routers = hop_matrix_routers.astype(int)

  return hop_matrix_routers

def create_adj_matrix_cores_to_routers( cores, routers ):
    adj_matrix_cores = np.zeros( (len(cores), len(routers)), dtype=int )

    for index in range(len(cores)) :
      adj_matrix_cores[cores[index], routers[index]] = 1

    return adj_matrix_cores

def create_hop_matrix_cores_tdg( tdg_matrix, hop_matrix_cores ):
  hop_matrix_cores_tdg = hop_matrix_cores.astype(int)

  for source_core_index in range(hop_matrix_cores.shape[0]):
    for destination_core_index in range(hop_matrix_cores.shape[1]):
      if tdg_matrix[source_core_index,destination_core_index] > 0:
        hop_matrix_cores_tdg[source_core_index,destination_core_index] = hop_matrix_cores[source_core_index,destination_core_index]
      else:
        hop_matrix_cores_tdg[source_core_index,destination_core_index] = 0

  return hop_matrix_cores_tdg

In [89]:
def create_hop_matrix_cores( hop_matrix_routers, adj_matrix_cores_to_routers ):
  hop_matrix_cores = np.zeros_like( adj_matrix_cores_to_routers )

  for source_core in range( adj_matrix_cores_to_routers.shape[0] ):
    for destination_core in range( adj_matrix_cores_to_routers.shape[1] ):
      if source_core == destination_core:
        hop_matrix_cores[source_core,destination_core] = 0
      else:
        source_router = list(adj_matrix_cores_to_routers[source_core]).index(1)
        destination_router = list(adj_matrix_cores_to_routers[destination_core]).index(1)
        hop_matrix_cores[source_core,destination_core] = hop_matrix_routers[source_router,destination_router]

  return hop_matrix_cores

def create_cost_routers_matrix( cost_cores_matrix, adj_matrix_cores_to_routers ):
  cost_routers_matrix = np.zeros_like( adj_matrix_cores_to_routers )

  for source_router in range( adj_matrix_cores_to_routers.shape[0] ):
    for destination_router in range( adj_matrix_cores_to_routers.shape[1] ):
      if source_router == destination_router:
        cost_routers_matrix[source_router,destination_router] = 0
      else:
        source_core = list(adj_matrix_cores_to_routers[:,source_router]).index(1)
        destination_core = list(adj_matrix_cores_to_routers[:,destination_router]).index(1)
        cost_routers_matrix[source_router,destination_router] = cost_cores_matrix[source_core,destination_core]

  return cost_routers_matrix


def create_cost_matrix( cores, routers, cost_cores_matrix ):
  adj_matrix_routers = np.array(
                      [[0, 1, 0, 1, 0, 0],
                      [1, 0, 1, 0, 1, 0],
                      [0, 1, 0, 0, 0, 1],
                      [1, 0, 0, 0, 1, 0],
                      [0, 1, 0, 1, 0, 1],
                      [0, 0, 1, 0, 1, 0]]
                      )

  adj_matrix_routers = np.array(
                     [[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                      [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                      [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                      [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
                      [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                      [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
                      [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0],
                      [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
                      [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]]
                      )
  hop_matrix_routers          = create_hop_matrix_routers( adj_matrix_routers )
  adj_matrix_cores_to_routers = create_adj_matrix_cores_to_routers( cores, routers )

  # Cores Vision
  hop_matrix_cores            = create_hop_matrix_cores( hop_matrix_routers, adj_matrix_cores_to_routers )
  cost_hop_matrix_cores       = cost_cores_matrix * hop_matrix_cores

  # Routers Vision
  cost_routers_matrix        = create_cost_routers_matrix( cost_cores_matrix, adj_matrix_cores_to_routers )
  cost_hop_matrix_routers    = cost_routers_matrix * hop_matrix_routers

  #
  '''
  print( "*-" * 20 )
  print( adj_matrix_routers )
  print( adj_matrix_cores_to_routers )
  print( "--" *20 )
  print( hop_matrix_routers )
  print( hop_matrix_cores )
  print( "--" *20 )
  print( cost_hop_matrix_routers )
  print( cost_hop_matrix_cores )
  print( "*-" * 20 )
  print( "\n" *2 )
  #'''

  return cost_hop_matrix_cores, cost_hop_matrix_routers


def swap_cores_between_routers( cores, routers, swaps ):
  for swap in swaps:
    router1 = swap[0]
    router2 = swap[1]
    # Verificar si los routers son válidos
    if router1 < 0 or router1 >= len(routers) or router2 < 0 or router2 >= len(routers):
      print("Los números de router no son válidos.")
      continue

    # Intercambiar los núcleos entre los routers
    temporal_core = cores[router1]
    cores[router1] = cores[router2]
    cores[router2] = temporal_core

  return cores, routers


def swap_cores_between_routers( cores, routers, swaps ):
  new_cores = cores.copy()  # Crear una copia de los núcleos originales
  new_routers = routers.copy()  # Crear una copia de los enrutadores originales

  for swap in swaps:
    router1 = swap[0]
    router2 = swap[1]
    # Verificar si los routers son válidos
    if router1 < 0 or router1 >= len(new_routers) or router2 < 0 or router2 >= len(new_routers):
      print("Los números de router no son válidos.")
      continue

    # Intercambiar los núcleos entre los routers en las copias
    temporal_core = new_cores[router1]
    new_cores[router1] = new_cores[router2]
    new_cores[router2] = temporal_core

  return new_cores, new_routers


## Ejemplo


In [90]:
## DSP
cost_matrix = np.array(
              [[0, 200,   0,    0,   0,   0],
               [0,   0, 200,  600,   0,   0],
               [0,   0,   0,    0,   0,   0],
               [0, 600,   0,    0, 200, 200],
               [0,   0,   0,  200,   0,   0],
               [0,   0,   0,  200,   0,   0]]
              )

# Particiones Iniciales		Particiones Finales
#    [0, 1, 2]	               {1, 3, 5}
#    [3, 4, 5]	               {0, 2, 4}


# Particiones Iniciales
#    [0, 1, 2]
#    [3, 4, 5]

cores   = [2,1,5,3,4,0]
routers = [0,1,2,3,4,5]

print("Núcleos antes del intercambio:", cores)
print("Routers antes del intercambio:", routers)

num_nodes = len(cores)

partition1 = [0,1,2]
partition2 = [3,4,5]



cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

print( "*-" * 20 )
print( "Vision Cores" )
print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
print( "--" *20 )
print( "Vision routers" )
print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
print( "*-" * 20 )
print( "\n" *2 )




# Ejemplo de uso

swaps =[(2,4)]
cores, routers = swap_cores_between_routers(cores, routers, swaps )


swaps =[(0,3)]
cores, routers = swap_cores_between_routers(cores, routers, swaps )


print("Núcleos después del intercambio:", cores)
print("Routers después del intercambio:", routers)



# Particiones Finales
#    {1, 3, 5}
#    {0, 2, 4}

partition1 = [0,1,2]
partition2 = [3,4,5]

cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

print( "*-" * 20 )
print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
print( "--" *20 )
print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
print( "*-" * 20 )
print( "\n" *2 )



Núcleos antes del intercambio: [2, 1, 5, 3, 4, 0]
Routers antes del intercambio: [0, 1, 2, 3, 4, 5]


ValueError: ignored

In [91]:

def calculate_gain_cost_matrix( cores, routers, partition1, partition2, cost_matrix ):

  gain_cost_matrix = np.full((num_nodes,num_nodes), 0)

  external_cost_matrix = create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)

  for i in range(num_nodes):
    for j in range(num_nodes):
      if external_adj_matrix[i][j] == 1:
        swap =[(i,j)]
        #print("Intercambio: ", swap)
        #print(  "*-" * 20 )
        #print( "Configuracion Inicial" )
        #print( routers )
        #print( cores )
        cores_swap, routers_swap = swap_cores_between_routers(cores, routers, swap )
        #print( "Configuracion Final" )
        #print( routers_swap )
        #print( cores_swap )
        #print(  "*-" * 20 )
        cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores_swap, routers_swap, cost_matrix )
        #print(  "*-" * 20 )
        #print( "Vision Routers" )
        #print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
        #print(  "--" *20 )
        #print(  "Vision Cores" )
        #print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
        #print(  "*-" * 20 )
        #print( "\n" *2 )
        difference_cost_vector =  calculate_difference_cost_vector( cost_hop_matrix_routers, partition1, partition2, num_nodes  )
        #print( "Diffenrence Vector" )
        #print( difference_cost_vector )
        gain_cost_matrix[i][j] = np.sum( cost_hop_matrix_routers ) #difference_cost_vector[i] + difference_cost_vector[j] - 2 * cost_matrix[i][j]
        #print( "Gain Cost" )
        #print( gain_cost_matrix[i][j]  )
        #print( "\n" *2 )

  return gain_cost_matrix



routers = [0,1,2,3,4,5]
cores   = [2,1,5,3,4,0]

routers = [0,1,2,3,4,5]
cores   = [2,5,1,0,4,3]

num_nodes = len(cores)

partition1 = [0,1,2]
partition2 = [3,4,5]

cost_matrix = np.array(
              [[0, 200,   0,    0,   0,   0],
               [0,   0, 200,  600,   0,   0],
               [0,   0,   0,    0,   0,   0],
               [0, 600,   0,    0, 200, 200],
               [0,   0,   0,  200,   0,   0],
               [0,   0,   0,  200,   0,   0]]
              )


#VOPD
#Cost: 7090
routers = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
cores   = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
num_nodes = len(cores)

partition1 = [0, 1, 2, 3, 4, 5, 6, 7]
partition2 = [8, 9,10,11,12,13,14,15]

#VOPD Best
#5127
#'''
#Cost: 5127
#routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#cores   = [0, 1, 2, 3, 10, 8, 9, 15, 14, 11, 7, 4, 12, 13, 5, 6]
#'''

# Optimal VOPD
#routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#cores   = [9, 8, 10, 14, 7, 6, 11, 12, 0, 5, 4, 13, 1, 2, 3, 15]

#VOPD Best
#Cost: 4468
routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
cores   = [0, 1, 2, 3, 7, 6, 5, 4, 8, 9, 10, 11, 12, 13, 14, 15]

#partition1 = [0, 1, 4, 5, 8, 9, 12, 13]
#partition2 = [2, 3,6,7,10,11,14,15]

#Cost: 10566
routers = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
cores   = [11,9,7,5,10,0,13,14,6,8,12,2,4,1,15,3]
num_nodes = len(cores)




cost_matrix = np.array(
   [[  0.,  70.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,  49.],
    [  0.,   0.,   0.,   0.,   0., 357.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0., 353.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0., 300.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 313., 500.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  94.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 313.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  16.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,  16.,   0.,   0.,  16.,   0.,   0.,   0.,  16.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 157.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 16.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  16.,   0.,  16.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,  27.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.] ]
)




def kernighan_lin( cores, routers, partition1, partition2, num_nodes ):
  print( "Costo Inicial" )
  cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

  '''
  print(  "*-" * 20 )
  print( "Vision Routers" )
  print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
  print(  "--" *20 )
  print(  "Vision Cores" )
  print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
  print(  "*-" * 20 )
  print( "\n" *2 )
  '''
  external_cost_matrix = create_external_cost_matrix( cost_hop_matrix_routers, partition1, partition2, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_hop_matrix_routers, partition1, partition2, num_nodes )

  '''
  print("*-"*20)
  print( "External Cost Matrix" )
  print( external_cost_matrix )


  print("--"*20)
  print( "Internal Cost Matrix" )
  print( internal_cost_matrix )
  print("*-"*20)
  '''

  cost_initial =  np.sum( cost_hop_matrix_routers )
  cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

  cost_best       = np.sum( cost_hop_matrix_routers )
  routers_best    = routers
  cores_best      = cores

  # Después de definir cost_best, routers_best y cores_best al comienzo del código
  consecutive_no_change_count = 0  # Inicializamos el contador en cero
  iteration_best = 0
  for iteration in range(1,11):



    next_moves = []
    #print("Iteracion #" + f"{iteration}")

    available_routers = [1 for _ in range(num_nodes)]
    intercambio = 1

    routers_inside = routers.copy()
    cores_inside   = cores.copy()

    while np.sum( available_routers ) > 0:
      #print( "Available routers: ", available_routers )
      #print( "Intercambio #" + f"{intercambio}"  )
      gain_cost_matrix = calculate_gain_cost_matrix( cores_inside, routers_inside, partition1, partition2, cost_matrix )
      #print( gain_cost_matrix )
      next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_routers )
      next_moves.append( next_move )

      #print( "\n" *1 )
      #print( "Antes del intercambio" )
      #print(  "*-" * 20 )
      #print( routers_inside )
      #print( cores_inside )

      #print( next_move )
      if next_move[0] == None:
        break
      swap = [(next_move[0],next_move[1])]

      cores_inside, routers_inside = swap_cores_between_routers(cores_inside, routers_inside, swap )


      #print(  "--" * 20 )
      #print( "Despues del intercambio" )
      #print( routers_inside )
      #print( cores_inside )
      #print( "\n" *2 )

      #print( "Costo" )
      cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores_inside, routers_inside, cost_matrix )
      #print(  "*-" * 20 )
      #print( "Vision Routers" )
      #print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
      #print(  "--" *20 )
      #print(  "Vision Cores" )
      #print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
      #print(  "*-" * 20 )
      #print( "\n" *2 )

      swap_routers        = update_swap_nodes( next_move )
      available_routers   = update_available_nodes( available_routers, swap_routers, num_nodes )
      intercambio += 1

    #print( next_moves )
    max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )
    #print( partial_next_moves )

    #print( "\n" *1 )
    #print( "Configuracion Inicial Iteracion" )
    #print(  "*-" * 20 )
    #print( routers )
    #print( cores )
    #print(  "*-" * 20 )

    cores, routers = swap_cores_between_routers( cores, routers, partial_next_moves )

    cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )


    cost_current = np.sum(cost_hop_matrix_routers)

    #print( "Configuracion Final Iteracion" )
    #print(  "*-" * 20 )
    #print( routers )
    #print( cores )
    #print(  "*-" * 20 )
    #print( "\n" *2 )

    if cost_current < cost_best:
      cost_best    = cost_current
      routers_best = routers
      cores_best   = cores
      iteration_best = iteration
      consecutive_no_change_count = 0

    else:
      consecutive_no_change_count += 1


    if consecutive_no_change_count >= 3:  # Si no hay cambios durante 3 iteraciones consecutivas
      break  # Terminar el bucle principal

  print("Best Configuration:")
  print("Iteration:", iteration_best)
  print("Cost Initial:", cost_best, " Cost Final: ", cost_initial )
  print("Routers:", routers_best)
  print("Cores:", cores_best)

  return cost_initial, iteration_best, cost_best, routers_best, cores_best



In [92]:
import random
# Optimal VOPD Cost: 4119
routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
cores   = [9, 8, 10, 14, 7, 6, 11, 12, 0, 5, 4, 13, 1, 2, 3, 15]

#Cost: 10566
routers = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
cores   = [11,9,7,5,10,0,13,14,6,8,12,2,4,1,15,3]
#Cost: 10566  Cost Final:  4125
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [9, 8, 10, 14, 7, 11, 12, 13, 6, 5, 2, 1, 15, 4, 3, 0]

#Cost: 7090
routers = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
cores   = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
#Cost: 4217  Cost Final:  4217
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [3, 2, 1, 0, 4, 5, 6, 7, 13, 12, 14, 9, 15, 11, 10, 8]

#Cost: 4468
routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
cores   = [0, 1, 2, 3, 7, 6, 5, 4, 8, 9, 10, 11, 12, 13, 14, 15]
#Cost: 4468  Cost Final:  4157
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [0, 1, 2, 3, 7, 6, 5, 4, 9, 12, 11, 15, 8, 13, 14, 10]



# Random
routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
cores   =  [9, 8, 10, 14, 7, 6, 11, 12, 0, 5, 4, 13, 1, 2, 3, 15]
random.shuffle(cores)

#[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#[11, 13, 5, 4, 2, 8, 12, 6, 15, 14, 10, 3, 1, 7, 0, 9]
#Cost: 9405  Cost Final:  4135
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [9, 8, 11, 12, 7, 6, 5, 13, 0, 10, 4, 14, 1, 2, 3, 15]

#[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#[8, 4, 0, 6, 14, 1, 12, 9, 2, 5, 7, 15, 13, 10, 3, 11]
#Cost: 11474  Cost Final:  4221
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [3, 4, 5, 10, 2, 15, 6, 11, 1, 12, 7, 14, 0, 13, 9, 8]

num_nodes = len(cores)

partition1 = [0, 1, 2, 3, 12, 13, 14, 15]
partition2 = [4, 5, 6, 7,  8,  9, 10, 11]

cost_initial_kl_1, iteration_kl_1, cost_kl_1, routers_kl_1, cores_kl_1 = kernighan_lin( cores, routers, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_1)
print("Cost:", cost_initial_kl_1, " Cost Final: ", cost_kl_1)
print("Routers:", routers_kl_1)
print("Cores:", cores_kl_1)


partition1 = [0, 1, 4, 5, 8, 9, 12, 13]
partition2 = [2, 3,6,7,10,11,14,15]

cost_initial_kl_2, iteration_kl_2, cost_kl_2, routers_kl_2, cores_kl_2  = kernighan_lin( cores_kl_1, routers_kl_1, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_2)
print("Cost:", cost_initial_kl_2, " Cost Final: ", cost_kl_2)
print("Routers:", routers_kl_2)
print("Cores:", cores_kl_2)

partition1 = [0, 1, 4, 5, 10, 11, 14, 15]
partition2 = [2, 3, 6, 7,  8,  9, 12, 13]

cost_initial_kl_3, iteration_kl_3, cost_kl_3, routers_kl_3, cores_kl_3  = kernighan_lin( cores_kl_2, routers_kl_2, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_3)
print("Cost:", cost_initial_kl_3, " Cost Final: ", cost_kl_3)
print("Routers:", routers_kl_3)
print("Cores:", cores_kl_3)

partition1 = [0, 1, 2, 3, 4, 5, 6, 7]
partition2 = [8, 9,10,11,12,13,14,15]


cost_initial_kl_4, iteration_kl_4, cost_kl_4,  routers_kl_4, cores_kl_4  = kernighan_lin( cores_kl_3, routers_kl_3, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_4)
print("Cost:", cost_initial_kl_4, " Cost Final: ", cost_kl_4)
print("Routers:", routers_kl_4)
print("Cores:", cores_kl_4)


partition1 = [0, 1, 2, 3,  8,  9, 10, 11]
partition2 = [4, 5, 6, 7, 12, 13, 14, 15]
cost_initial_kl_5, iteration_kl_5, cost_kl_5,  routers_kl_5, cores_kl_5  = kernighan_lin( cores_kl_4, routers_kl_4, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_5)
print("Cost:", cost_initial_kl_5, " Cost Final: ", cost_kl_5)
print("Routers:", routers_kl_5)
print("Cores:", cores_kl_5)


partition1 = [0, 2, 4, 6, 8, 10, 12, 14]
partition2 = [1, 3, 5, 7, 9, 11, 13, 15]
cost_initial_kl_6, iteration_kl_6, cost_kl_6,  routers_kl_6, cores_kl_6  = kernighan_lin( cores_kl_5, routers_kl_5, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_6)
print("Cost:", cost_initial_kl_6, " Cost Final: ", cost_kl_6)
print("Routers:", routers_kl_6)
print("Cores:", cores_kl_6)

Costo Inicial
Best Configuration:
Iteration: 5
Cost Initial: 4686  Cost Final:  8050
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [12, 14, 13, 15, 6, 5, 4, 3, 7, 11, 2, 1, 9, 8, 10, 0]
Best Configuration:
Iteration: 5
Cost: 8050  Cost Final:  4686
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [12, 14, 13, 15, 6, 5, 4, 3, 7, 11, 2, 1, 9, 8, 10, 0]
Costo Inicial
Best Configuration:
Iteration: 1
Cost Initial: 4529  Cost Final:  4686
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [12, 13, 14, 15, 6, 5, 4, 3, 7, 11, 2, 1, 9, 8, 10, 0]
Best Configuration:
Iteration: 1
Cost: 4686  Cost Final:  4529
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [12, 13, 14, 15, 6, 5, 4, 3, 7, 11, 2, 1, 9, 8, 10, 0]
Costo Inicial
Best Configuration:
Iteration: 4
Cost Initial: 4425  Cost Final:  4529
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [12, 13, 1, 2, 6, 5, 4, 3, 7, 11, 15, 0, 9, 

In [94]:
print("Routers:", routers)
print("Cores:", cores)

print("Best Configuration 1:")
print("Iteration:", iteration_kl_1)
print("Cost:", cost_initial_kl_1, " Cost Final: ", cost_kl_1)
print("Routers:", routers_kl_1)
print("Cores:", cores_kl_1)
print("\n")

print("Best Configuration 2:")
print("Iteration:", iteration_kl_2)
print("Cost:", cost_initial_kl_2, " Cost Final: ", cost_kl_2)
print("Routers:", routers_kl_2)
print("Cores:", cores_kl_2)
print("\n")

print("Best Configuration 3:")
print("Iteration:", iteration_kl_3)
print("Cost:", cost_initial_kl_3, " Cost Final: ", cost_kl_3)
print("Routers:", routers_kl_3)
print("Cores:", cores_kl_3)
print("\n")

print("Best Configuration 4:")
print("Iteration:", iteration_kl_4)
print("Cost:", cost_initial_kl_4, " Cost Final: ", cost_kl_4)
print("Routers:", routers_kl_4)
print("Cores:", cores_kl_4)
print("\n")

print("Best Configuration 5:")
print("Iteration:", iteration_kl_5)
print("Cost:", cost_initial_kl_5, " Cost Final: ", cost_kl_5)
print("Routers:", routers_kl_5)
print("Cores:", cores_kl_5)
print("\n")

print("Best Configuration 6:")
print("Iteration:", iteration_kl_6)
print("Cost:", cost_initial_kl_6, " Cost Final: ", cost_kl_6)
print("Routers:", routers_kl_6)
print("Cores:", cores_kl_6)
print("\n")

Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [12, 6, 2, 14, 5, 3, 11, 13, 7, 9, 10, 1, 4, 8, 0, 15]
Best Configuration 1:
Iteration: 5
Cost: 8050  Cost Final:  4686
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [12, 14, 13, 15, 6, 5, 4, 3, 7, 11, 2, 1, 9, 8, 10, 0]


Best Configuration 2:
Iteration: 1
Cost: 4686  Cost Final:  4529
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [12, 13, 14, 15, 6, 5, 4, 3, 7, 11, 2, 1, 9, 8, 10, 0]


Best Configuration 3:
Iteration: 4
Cost: 4529  Cost Final:  4425
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [12, 13, 1, 2, 6, 5, 4, 3, 7, 11, 15, 0, 9, 8, 10, 14]


Best Configuration 4:
Iteration: 2
Cost: 4425  Cost Final:  4371
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [10, 14, 1, 2, 11, 15, 0, 3, 7, 6, 5, 4, 9, 8, 12, 13]


Best Configuration 5:
Iteration: 2
Cost: 4371  Cost Final:  4221
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 

## Kernighan-Lin

In [ ]:
def kernighan_lin( cores, routers, partition1, partition2, cost_matrix ):

  number_of_iteration   = 1
  number_of_interchange = 1

  for number_of_iteration in range(10):
    next_moves = []
    print( "\n"*2 )
    print( "### Iteración "+ f"{number_of_iteration}" + "###"  )

    routers_intermedio = routers.copy()
    cores_intermedio = cores.copy()

    available_routers = [1 for _ in range(num_nodes)]

    number_of_interchange = 0

    while np.sum(available_routers) :
      identation = " "*4
      level = 0
      print( identation*level +"Particiones Iniciales" )
      print(identation*level + "Routers antes del intercambio:", routers_intermedio)
      print(identation*level + "Núcleos antes del intercambio:", cores_intermedio)
      level = 1
      print(identation*level + "Particion 1" )
      print(identation*level +"Routers antes del intercambio:", [routers_intermedio[i] for i in partition1] )
      print(identation*level +"Núcleos antes del intercambio:", [cores_intermedio[i] for i in partition1])
      print(identation*level +"Particion 2" )
      print(identation*level +"Routers antes del intercambio:", [routers_intermedio[i] for i in partition2])
      print(identation*level +"Núcleos antes del intercambio:", [cores_intermedio[i] for i in partition2])

      cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores_intermedio, routers_intermedio, cost_matrix )

      print( identation*level + "*-" * 20 )
      print( identation*level +"Vision Routers" )
      print( identation*level, cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
      print( identation*level + "--" *20 )
      print(identation*level + "Vision Cores" )
      print( identation*level , cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
      print( identation*level + "*-" * 20 )
      print( "\n" *2 )

      level = 2
      print( identation*level + "# Intercambio #" + f"{number_of_interchange}" )


      difference_cost_vector =  calculate_difference_cost_vector( cost_hop_matrix_routers, partition1, partition2, num_nodes  )
      gain_cost_matrix       =  calculate_gain_cost_matrix( difference_cost_vector, partition1, partition2, None, cost_matrix )
      next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_routers )



      if next_move[0] == None:
        break

      swap =[(next_move[0],next_move[1])]

      cores_intermedio, routers_intermedio = swap_cores_between_routers(cores_intermedio, routers_intermedio, swap )

      swap_routers        = update_swap_nodes( next_move )
      available_routers   = update_available_nodes( available_routers, swap_routers )
      next_moves.append( next_move )

      level = 0
      print( identation*level +"Particiones Intermedias" )
      print(identation*level + "Routers despues del intercambio:", routers_intermedio)
      print(identation*level + "Núcleos despues del intercambio:", cores_intermedio)
      level = 1
      print(identation*level + "Particion 1" )
      print(identation*level +"Routers despues del intercambio:", [routers_intermedio[i] for i in partition1] )
      print(identation*level +"Núcleos despues del intercambio:", [cores_intermedio[i] for i in partition1])
      print(identation*level +"Particion 2" )
      print(identation*level +"Routers despues del intercambio:", [routers_intermedio[i] for i in partition2])
      print(identation*level +"Núcleos despues del intercambio:", [cores_intermedio[i] for i in partition2])


      cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores_intermedio, routers_intermedio, cost_matrix )

      print( identation*level + "*-" * 20 )
      print( identation*level +"Vision Routers" )
      print( identation*level , cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
      print( identation*level + "--" *20 )
      print(identation*level + "Vision Cores" )
      print( identation*level , cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
      print( identation*level + "*-" * 20 )
      print( "\n" *2 )





      number_of_interchange += 1

      #'''
      print("-"*40)
      print("Difference cost Vector")
      print( difference_cost_vector )
      print("-"*40)

      print("\n")

      print("-"*40)
      print("Gain Cost Matrix")
      print( gain_cost_matrix )
      print("-"*40)

      print("\n")

      print("-"*40)
      print("Particiones Potenciales")
      print( "Movimiento Potencial:", next_move )
      print( "Partition1_potencial:", partition1 )
      print( "Partition2_potencial:", partition2 )
      #'''


    max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )

    print(max_partial_sum, partial_next_moves)

    print( "Movimientos Potenciales: ", next_moves )





    print( "Movimientos Finales:", partial_next_moves )


    if max_partial_sum <= 0:
      print("Terminando debido a que max_partial_sum es igual o menor a 0")
      level = 1
      print(identation*level + "Routers antes del intercambio:", routers)
      print(identation*level + "Núcleos antes del intercambio:", cores)
      level = 2
      print( identation*level + "Particion 1" )
      print(identation*level +"Routers antes del intercambio:", [routers[i] for i in partition1] )
      print(identation*level +"Núcleos antes del intercambio:", [cores[i] for i in partition1])
      print(identation*level +"Particion 2" )
      print(identation*level +"Routers antes del intercambio:", [routers[i] for i in partition2])
      print(identation*level +"Núcleos antes del intercambio:", [cores[i] for i in partition2])

      return partition1, partition2


    partition1_post, partition2_post = swap_cores( partial_next_moves, partition1, partition2 )

    level = 0
    print( identation*level +"Particiones Iniciales" )
    print(identation*level + "Routers antes del intercambio:", routers)
    print(identation*level + "Núcleos antes del intercambio:", cores)
    cores, routers = swap_cores_between_routers(cores, routers, partial_next_moves )
    print( identation*level +"Particiones Finales" )
    print(identation*level + "Routers antes del intercambio:", routers)
    print(identation*level + "Núcleos antes del intercambio:", cores)

    cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

    print( "*-" * 20 )
    print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
    print( "--" *20 )
    print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
    print( "*-" * 20 )
    print( "\n" *2 )





    if partition1_post == partition2 and partition2_post == partition1:
      print( "Particiones Finales Iteracion" )
      print( partition1, partition2_post )
      print( partition2, partition1_post )
      return partition1, partition2






In [ ]:

routers = [0,1,2,3,4,5]
cores   = [2,1,5,3,4,0]

kernighan_lin( cores, routers, partition1, partition2, cost_matrix )

print(  "\n" )
print( "Particiones Iniciales" + "\t"*2 + "Particiones Finales" )
#print(  f"{partition1}" + "\t"*1 + f"{partition1_kl}" )
#print(  f"{partition2}" + "\t"*1 + f"{partition2_kl}" )




### Iteración 0###
Particiones Iniciales
Routers antes del intercambio: [0, 1, 2, 3, 4, 5]
Núcleos antes del intercambio: [2, 1, 5, 3, 4, 0]
    Particion 1
    Routers antes del intercambio: [0, 1, 2]
    Núcleos antes del intercambio: [2, 1, 5]
    Particion 2
    Routers antes del intercambio: [3, 4, 5]
    Núcleos antes del intercambio: [3, 4, 0]
    *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
    Vision Routers
     [[   0    0    0    0    0    0]
 [ 200    0    0 1200    0    0]
 [   0    0    0  600    0    0]
 [   0 1200  600    0  200    0]
 [   0    0    0  200    0    0]
 [   0  400    0    0    0    0]] 4600
    ----------------------------------------
    Vision Cores
     [[   0  400    0    0    0    0]
 [   0    0  200 1200    0    0]
 [   0    0    0    0    0    0]
 [   0 1200    0    0  200  600]
 [   0    0    0  200    0    0]
 [   0    0    0  600    0    0]] 4600
    *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-



        # Intercambio #0
*-*-*-*-*-*-*-*-*-*-*-*-*-*

TypeError: ignored

# Version #*4*

## Funciones

In [10]:
import numpy as np
import math

def create_external_adj_matrix(partition1, partition2, num_nodes):
  external_adj_matrix = np.zeros((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      external_adj_matrix[node1][node2] = 1
      external_adj_matrix[node2][node1] = 1

  return external_adj_matrix


def create_internal_adj_matrix( partition1, partition2, num_nodes ):

  internal_adj_matrix = np.ones((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      internal_adj_matrix[node1][node2] = 0
      internal_adj_matrix[node2][node1] = 0

  return internal_adj_matrix

def create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes ):
  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
  external_cost_matrix = np.zeros((num_nodes,num_nodes))

  for i in range(num_nodes):
    for j in range(num_nodes):
      external_cost_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

  return external_cost_matrix


def create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes ):

  internal_adj_matrix = create_internal_adj_matrix(partition1, partition2, num_nodes)
  internal_cost_matrix = np.zeros((num_nodes,num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
      if i != j:
        internal_cost_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

  return internal_cost_matrix


def calculate_difference_cost_vector( cost_matrix, partition1, partition2, num_nodes ):

  external_cost_matrix = create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes )


  difference_cost_matrix = np.zeros((num_nodes, num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
     difference_cost_matrix[i][j] += external_cost_matrix[j][i] - internal_cost_matrix[j][i]

  #'''
  print("*-"*20)
  print( "External Cost Matrix" )
  print( external_cost_matrix )


  print("--"*20)
  print( "Internal Cost Matrix" )
  print( internal_cost_matrix )

  #print("--"*20)
  #print( "Externa-Internal Cost Matrix" )
  #print( difference_cost_matrix )
  print("*-" *20)
  print("\n")
  #'''
  #return np.dot([1,1,1,1,1,1],difference_cost_matrix)
  #return np.dot(np.array(num_nodes), difference_cost_matrix)
  return np.dot([1 for _ in range(num_nodes)], difference_cost_matrix)

def update_difference_cost_vector( cost_matrix, difference_cost_vector, partition1, partition2, available_nodes, swap_nodes  ):

    internal_adj_matrix = create_internal_adj_matrix( partition1, partition2, num_nodes)
    difference_cost_internal_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_internal_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

    external_adj_matrix = create_external_adj_matrix( partition1, partition2, num_nodes)
    difference_cost_external_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
          difference_cost_external_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

    difference_cost_prime_matrix = np.zeros((num_nodes,num_nodes))
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_prime_matrix[i][j] = difference_cost_internal_matrix[i][j] - difference_cost_external_matrix[i][j]

    difference_cost_vector          = np.multiply(available_nodes, difference_cost_vector)
    difference_cost_internal_vector = np.dot(swap_nodes, difference_cost_internal_matrix)
    difference_cost_external_vector = np.dot(swap_nodes, difference_cost_external_matrix)
    difference_cost_prime_vector    = difference_cost_vector + 2 * difference_cost_internal_vector - 2 * difference_cost_external_vector

    #'''
    print(internal_adj_matrix)
    print("*-"*20)
    print( "Difference Cost Internal Matrix")
    for row in difference_cost_internal_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost Internal Vector")
    print( difference_cost_internal_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost External Matrix")
    for row in difference_cost_external_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost External Vector")
    print( difference_cost_external_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost Vector")
    print( difference_cost_vector)

    print("-"*40)

    print( "Difference Cost Updated Vector")
    print( difference_cost_prime_vector )

    print("*-"*20)
    print("\n")
    #'''
    return difference_cost_prime_vector

def calculate_gain_cost_matrix( difference_cost_vector, partition1, partition2 ,external_adj_matrix, cost_matrix ):
  gain_cost_matrix = np.full((num_nodes,num_nodes), 0)

  external_cost_matrix = create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)

  for i in range(num_nodes):
    for j in range(num_nodes):
      if external_adj_matrix[i][j] == 1:
        gain_cost_matrix[i][j] = difference_cost_vector[i] + difference_cost_vector[j] - 2 * cost_matrix[i][j]

  return gain_cost_matrix


def swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes ):
  partition1_copy = set(partition1)
  partition2_copy = set(partition2)
  max_gain = float('inf')
  max_i, max_j = None, None

  for i in range( num_nodes ):
    for j in range( num_nodes ):
      if gain_cost_matrix[i][j] < max_gain  and gain_cost_matrix[i][j] != 0 and available_nodes[i] == 1 and available_nodes[j] == 1 and i in partition1_copy and j in partition2_copy:
        max_gain = gain_cost_matrix[i][j]
        max_i = i
        max_j = j

  #print( max_i, max_j, max_gain )
  next_move = [ max_i, max_j, max_gain]

  for ci in partition1_copy:
    for cj in partition2_copy:
      if max_i== ci and max_j == cj:
        partition1_copy.remove(max_i)
        partition2_copy.remove(max_j)
        partition1_copy.add(max_j)
        partition2_copy.add(max_i)
        #print("Hubo intercambio Parcial!!!!!", (max_i, max_j))

  return next_move

def max_subarray_sum(next_moves):
    select_next_moves = []
    current_sum = 0
    max_sum = np.inf

    for next_move in next_moves:
      if math.isfinite(next_move[2]):
        current_sum = int(next_move[2])
        #print( next_move, current_sum )
        if current_sum < max_sum:
          max_sum = current_sum
          select_next_moves.append( next_move )

    return max_sum, select_next_moves

def swap_cores(  partial_next_moves, partition1, partition2  ):
  for next_move in partial_next_moves:
    for ci in partition1:
     for cj in partition2:
        if ci == next_move[0] and cj == next_move[1]:
          partition1.remove(ci)
          partition2.remove(cj)
          partition1.add(cj)
          partition2.add(ci)
  return partition1, partition2

def swap_cores(partial_next_moves, partition1, partition2):
    new_partition1 = set(partition1)
    new_partition2 = set(partition2)

    for next_move in partial_next_moves:
        node_from = next_move[0]
        node_to = next_move[1]

        if node_from in new_partition1 and node_to in new_partition2:
            new_partition1.remove(node_from)
            new_partition2.remove(node_to)
            new_partition1.add(node_to)
            new_partition2.add(node_from)
            #print( "Hubo intercambio Final!!!!!" + f"{node_from,node_to}"  )

    return new_partition1, new_partition2

def update_available_nodes( available_nodes, swap_nodes, num_nodes ):
  return  [a ^ b for a, b in zip(available_nodes, swap_nodes)]

def update_swap_nodes( next_move ):
  indices = next_move[0:2]
  swap_nodes = [1 if i in indices else 0 for i in range(num_nodes)]
  return swap_nodes


## Topology routers

In [11]:
import numpy as np

def create_hop_matrix_routers( adj_matrix_routers ):
  hop_matrix_routers = adj_matrix_routers.astype(float)
  hop_matrix_routers[hop_matrix_routers == 0] = np.inf

  for intermediate_router in range( adj_matrix_routers.shape[1] ):
    for source_router in range( adj_matrix_routers.shape[0] ):
      for destination_router in range( adj_matrix_routers.shape[1] ):
        if source_router == destination_router:
          hop_matrix_routers[source_router, destination_router] = 0
        else:
          hop_matrix_routers[source_router, destination_router]  = min(
              hop_matrix_routers[source_router, destination_router],
              hop_matrix_routers[source_router, intermediate_router] + hop_matrix_routers[intermediate_router, destination_router]
          )
  hop_matrix_routers = hop_matrix_routers.astype(int)

  return hop_matrix_routers

def create_adj_matrix_cores_to_routers( cores, routers ):
    adj_matrix_cores = np.zeros( (len(cores), len(routers)), dtype=int )

    for index in range(len(cores)) :
      adj_matrix_cores[cores[index], routers[index]] = 1

    return adj_matrix_cores

def create_hop_matrix_cores_tdg( tdg_matrix, hop_matrix_cores ):
  hop_matrix_cores_tdg = hop_matrix_cores.astype(int)

  for source_core_index in range(hop_matrix_cores.shape[0]):
    for destination_core_index in range(hop_matrix_cores.shape[1]):
      if tdg_matrix[source_core_index,destination_core_index] > 0:
        hop_matrix_cores_tdg[source_core_index,destination_core_index] = hop_matrix_cores[source_core_index,destination_core_index]
      else:
        hop_matrix_cores_tdg[source_core_index,destination_core_index] = 0

  return hop_matrix_cores_tdg

In [72]:
def create_hop_matrix_cores( hop_matrix_routers, adj_matrix_cores_to_routers ):
  hop_matrix_cores = np.zeros_like( adj_matrix_cores_to_routers )

  for source_core in range( adj_matrix_cores_to_routers.shape[0] ):
    for destination_core in range( adj_matrix_cores_to_routers.shape[1] ):
      if source_core == destination_core:
        hop_matrix_cores[source_core,destination_core] = 0
      else:
        source_router = list(adj_matrix_cores_to_routers[source_core]).index(1)
        destination_router = list(adj_matrix_cores_to_routers[destination_core]).index(1)
        hop_matrix_cores[source_core,destination_core] = hop_matrix_routers[source_router,destination_router]

  return hop_matrix_cores

def create_cost_routers_matrix( cost_cores_matrix, adj_matrix_cores_to_routers ):
  cost_routers_matrix = np.zeros_like( adj_matrix_cores_to_routers )

  for source_router in range( adj_matrix_cores_to_routers.shape[0] ):
    for destination_router in range( adj_matrix_cores_to_routers.shape[1] ):
      if source_router == destination_router:
        cost_routers_matrix[source_router,destination_router] = 0
      else:
        source_core = list(adj_matrix_cores_to_routers[:,source_router]).index(1)
        destination_core = list(adj_matrix_cores_to_routers[:,destination_router]).index(1)
        cost_routers_matrix[source_router,destination_router] = cost_cores_matrix[source_core,destination_core]

  return cost_routers_matrix


def create_cost_matrix( cores, routers, cost_cores_matrix ):
  adj_matrix_routers = np.array(
                      [[0, 1, 0, 1, 0, 0],
                      [1, 0, 1, 0, 1, 0],
                      [0, 1, 0, 0, 0, 1],
                      [1, 0, 0, 0, 1, 0],
                      [0, 1, 0, 1, 0, 1],
                      [0, 0, 1, 0, 1, 0]]
                      )
  #'''
  adj_matrix_routers = np.array(
                     [[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                      [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                      [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                      [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
                      [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                      [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
                      [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0],
                      [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
                      [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]]
                      )
  #'''
  hop_matrix_routers          = create_hop_matrix_routers( adj_matrix_routers )
  adj_matrix_cores_to_routers = create_adj_matrix_cores_to_routers( cores, routers )

  # Cores Vision
  hop_matrix_cores            = create_hop_matrix_cores( hop_matrix_routers, adj_matrix_cores_to_routers )
  cost_hop_matrix_cores       = cost_cores_matrix * hop_matrix_cores

  # Routers Vision
  cost_routers_matrix        = create_cost_routers_matrix( cost_cores_matrix, adj_matrix_cores_to_routers )
  cost_hop_matrix_routers    = cost_routers_matrix * hop_matrix_routers

  #
  '''
  print( "*-" * 20 )
  print( adj_matrix_routers )
  print( adj_matrix_cores_to_routers )
  print( "--" *20 )
  print( hop_matrix_routers )
  print( hop_matrix_cores )
  print( "--" *20 )
  print( cost_hop_matrix_routers )
  print( cost_hop_matrix_cores )
  print( "*-" * 20 )
  print( "\n" *2 )
  #'''

  return cost_hop_matrix_cores, cost_hop_matrix_routers


def swap_cores_between_routers( cores, routers, swaps ):
  for swap in swaps:
    router1 = swap[0]
    router2 = swap[1]
    # Verificar si los routers son válidos
    if router1 < 0 or router1 >= len(routers) or router2 < 0 or router2 >= len(routers):
      print("Los números de router no son válidos.")
      continue

    # Intercambiar los núcleos entre los routers
    temporal_core = cores[router1]
    cores[router1] = cores[router2]
    cores[router2] = temporal_core

  return cores, routers


def swap_cores_between_routers( cores, routers, swaps ):
  new_cores = cores.copy()  # Crear una copia de los núcleos originales
  new_routers = routers.copy()  # Crear una copia de los enrutadores originales

  for swap in swaps:
    router1 = swap[0]
    router2 = swap[1]
    # Verificar si los routers son válidos
    if router1 < 0 or router1 >= len(new_routers) or router2 < 0 or router2 >= len(new_routers):
      print("Los números de router no son válidos.")
      continue

    # Intercambiar los núcleos entre los routers en las copias
    temporal_core = new_cores[router1]
    new_cores[router1] = new_cores[router2]
    new_cores[router2] = temporal_core

  return new_cores, new_routers


## Ejemplo


In [73]:
## DSP
cost_matrix = np.array(
              [[0, 200,   0,    0,   0,   0],
               [0,   0, 200,  600,   0,   0],
               [0,   0,   0,    0,   0,   0],
               [0, 600,   0,    0, 200, 200],
               [0,   0,   0,  200,   0,   0],
               [0,   0,   0,  200,   0,   0]]
              )

# Particiones Iniciales		Particiones Finales
#    [0, 1, 2]	               {1, 3, 5}
#    [3, 4, 5]	               {0, 2, 4}


# Particiones Iniciales
#    [0, 1, 2]
#    [3, 4, 5]


routers = [0,1,2,3,4,5]
cores   = [1,0,2,4,3,5]

print("Routers antes del intercambio:", routers)
print("Núcleos antes del intercambio:", cores)


num_nodes = len(cores)

partition1 = [0,1,2]
partition2 = [3,4,5]

#partition1 = [0,3,4]
#partition2 = [1,2,5]



cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

print( "*-" * 20 )
print( "Vision Routers" )
print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
print( "--" *20 )
print( "Vision Cores" )
print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
print( "*-" * 20 )
print( "\n" *2 )

difference_cost_vector =  calculate_difference_cost_vector( cost_hop_matrix_routers, partition1, partition2, num_nodes  )
print( "Diffenrence Vector" )
print( difference_cost_vector )
print( "\n" *2 )

gain_cost_matrix = calculate_gain_cost_matrix( cores, routers, partition1, partition2, cost_matrix )
print( "Gain Matrix Outside" )
print( gain_cost_matrix , np.sum( cost_hop_matrix_routers ) )
print( "\n" *2 )




# Ejemplo de uso

swaps =[(4,5)]
cores, routers = swap_cores_between_routers(cores, routers, swaps )


#swaps =[(0,3)]
#cores, routers = swap_cores_between_routers(cores, routers, swaps )


print("Núcleos después del intercambio:", cores)
print("Routers después del intercambio:", routers)



# Particiones Finales
#    {1, 3, 5}
#    {0, 2, 4}


cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

print( "*-" * 20 )
print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
print( "--" *20 )
print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
print( "*-" * 20 )
print( "\n" *2 )



Routers antes del intercambio: [0, 1, 2, 3, 4, 5]
Núcleos antes del intercambio: [1, 0, 2, 4, 3, 5]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Vision Routers
[[  0   0 200   0 600   0]
 [200   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [  0   0   0   0 200   0]
 [600   0   0 200   0 200]
 [  0   0   0   0 200   0]] 2400
----------------------------------------
Vision Cores
[[  0 200   0   0   0   0]
 [  0   0 400 600   0   0]
 [  0   0   0   0   0   0]
 [  0 600   0   0 800 200]
 [  0   0   0 800   0   0]
 [  0   0   0 200   0   0]] 3800
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-



*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
External Cost Matrix
[[  0.   0.   0.   0. 600.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [600.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]]
----------------------------------------
Internal Cost Matrix
[[  0.   0. 200.   0.   0.   0.]
 [200.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.

In [81]:
def calculate_gain_cost_matrix( cores, routers, partition1, partition2, cost_matrix ):

  gain_cost_matrix = np.full((num_nodes,num_nodes), 0)

  external_cost_matrix = create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)

  cost_hop_matrix_cores_outside, cost_hop_matrix_routers_outside = create_cost_matrix( cores, routers, cost_matrix )

  print(  "*-" * 20 )
  print( "Vision Routers Outside" )
  print( cost_hop_matrix_routers_outside, np.sum( cost_hop_matrix_routers_outside ) )
  print(  "*-" * 20 )
  print( "\n" *2 )

  difference_cost_vector =  calculate_difference_cost_vector( cost_hop_matrix_routers_outside, partition1, partition2, num_nodes  )
  print( "Diffenrence Vector" )
  print( difference_cost_vector )
  print( "\n" *2 )

  gain_cost_matrix_outside = np.full((num_nodes,num_nodes), 0)
  for i in range(num_nodes):
    for j in range(num_nodes):
      if external_adj_matrix[i][j] == 1:
        gain_cost_matrix_outside[i][j] = difference_cost_vector[i] + difference_cost_vector[j] - 0 * cost_hop_matrix_routers_outside[i][j]

  print( "Externa-Internal Matrix " )
  print( gain_cost_matrix_outside  )
  print( "\n" *2 )


  for i in range(num_nodes):
    for j in range(num_nodes):
      if external_adj_matrix[i][j] == 1:
        gain_cost_matrix_outside[i][j] += - 1* cost_hop_matrix_routers_outside[i][j] - 1* cost_hop_matrix_routers_outside[j][i]



  for i in range(num_nodes):
    for j in range(num_nodes):
      if external_adj_matrix[i][j] == 1:
        swap =[(i,j)]
        #print("Intercambio: ", swap)
        #print(  "*-" * 20 )
        #print( "Configuracion Inicial" )
        #print( routers )
        #print( cores )
        cores_swap, routers_swap = swap_cores_between_routers(cores, routers, swap )
        #print( "Configuracion Final" )
        #print( routers_swap )
        #print( cores_swap )
        #print(  "*-" * 20 )
        cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores_swap, routers_swap, cost_matrix )
        #print(  "*-" * 20 )
        #print( "Vision Routers" )
        #print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
        #print(  "--" *20 )
        #print(  "Vision Cores" )
        #print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
        #print(  "*-" * 20 )
        #print( "\n" *2 )
        #difference_cost_vector =  calculate_difference_cost_vector( cost_hop_matrix_routers, partition1, partition2, num_nodes  )
        #print( "Diffenrence Vector" )
        #print( difference_cost_vector )
        gain_cost_matrix[i][j] = np.sum( cost_hop_matrix_routers ) #difference_cost_vector[i] + difference_cost_vector[j] - 2 * cost_matrix[i][j]
        #print( "Gain Cost" )
        #print( gain_cost_matrix[i][j]  )
        #print( "\n" *2 )

  return gain_cost_matrix



def swap_cores_partition( partition1, partition2, gain_cost_matrix, cost_comm,available_nodes ):
  partition1_copy = set(partition1)
  partition2_copy = set(partition2)
  max_gain = - float('inf')
  max_i, max_j = None, None

  for i in range( num_nodes ):
    for j in range( num_nodes ):
      #and gain_cost_matrix[i][j] != 0
      if max_gain < gain_cost_matrix[i][j]  and  available_nodes[i] == 1 and available_nodes[j] == 1: #and i in partition1_copy and j in partition2_copy:
        max_gain = gain_cost_matrix[i][j]
        max_i = i
        max_j = j

  #print( max_i, max_j, max_gain )
  next_move = [ max_i, max_j, max_gain, cost_comm]

  for ci in partition1_copy:
    for cj in partition2_copy:
      if max_i== ci and max_j == cj:
        partition1_copy.remove(max_i)
        partition2_copy.remove(max_j)
        partition1_copy.add(max_j)
        partition2_copy.add(max_i)
        #print("Hubo intercambio Parcial!!!!!", (max_i, max_j))

  return next_move

def max_subarray_sum(next_moves):
  potential_next_moves = []
  select_next_moves = []
  current_sum = 0
  max_sum = -np.inf

  for next_move in next_moves:
    if math.isfinite(next_move[2]):
      potential_next_moves.append( next_move )
      current_sum += int(next_move[2])
      print( next_move, max_sum, current_sum )
      if max_sum <= current_sum:
        max_sum = current_sum
        select_next_moves.extend( potential_next_moves )
        potential_next_moves = []

  return max_sum, select_next_moves






routers = [0,1,2,3,4,5]
cores   = [2,1,5,3,4,0]

routers = [0,1,2,3,4,5]
cores   = [2,5,1,0,4,3]

num_nodes = len(cores)

partition1 = [0,1,2]
partition2 = [3,4,5]

cost_matrix = np.array(
              [[0, 200,   0,    0,   0,   0],
               [0,   0, 200,  600,   0,   0],
               [0,   0,   0,    0,   0,   0],
               [0, 600,   0,    0, 200, 200],
               [0,   0,   0,  200,   0,   0],
               [0,   0,   0,  200,   0,   0]]
              )


#VOPD
#Cost: 7090
#routers = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
#cores   = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
#num_nodes = len(cores)

#partition1 = [0, 1, 2, 3, 4, 5, 6, 7]
#partition2 = [8, 9,10,11,12,13,14,15]

#VOPD Best
#5127
#'''
#Cost: 5127
#routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#cores   = [0, 1, 2, 3, 10, 8, 9, 15, 14, 11, 7, 4, 12, 13, 5, 6]
#'''

# Optimal VOPD
#routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#cores   = [9, 8, 10, 14, 7, 6, 11, 12, 0, 5, 4, 13, 1, 2, 3, 15]

#VOPD Best
#Cost: 4468
#routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#cores   = [0, 1, 2, 3, 7, 6, 5, 4, 8, 9, 10, 11, 12, 13, 14, 15]

#partition1 = [0, 1, 4, 5, 8, 9, 12, 13]
#partition2 = [2, 3,6,7,10,11,14,15]

#Cost: 10566
#routers = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
#cores   = [11,9,7,5,10,0,13,14,6,8,12,2,4,1,15,3]
#num_nodes = len(cores)



#'''
cost_matrix = np.array(
   [[  0.,  70.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,  49.],
    [  0.,   0.,   0.,   0.,   0., 357.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0., 353.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0., 300.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 313., 500.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  94.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 313.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  16.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,  16.,   0.,   0.,  16.,   0.,   0.,   0.,  16.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 157.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 16.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  16.,   0.,  16.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,  27.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.] ]
)
#'''



def kernighan_lin( cores, routers, partition1, partition2, num_nodes ):
  print( "Costo Inicial" )
  cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

  '''
  print(  "*-" * 20 )
  print( "Vision Routers" )
  print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
  print(  "--" *20 )
  print(  "Vision Cores" )
  print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
  print(  "*-" * 20 )
  print( "\n" *2 )
  '''
  external_cost_matrix = create_external_cost_matrix( cost_hop_matrix_routers, partition1, partition2, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_hop_matrix_routers, partition1, partition2, num_nodes )

  '''
  print("*-"*20)
  print( "External Cost Matrix" )
  print( external_cost_matrix )


  print("--"*20)
  print( "Internal Cost Matrix" )
  print( internal_cost_matrix )
  print("*-"*20)
  '''

  cost_initial =  np.sum( cost_hop_matrix_routers )
  cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

  cost_best       = np.sum( cost_hop_matrix_routers )
  routers_best    = routers
  cores_best      = cores

  # Después de definir cost_best, routers_best y cores_best al comienzo del código
  consecutive_no_change_count = 0  # Inicializamos el contador en cero
  iteration_best = 0
  for iteration in range(1,11):

    next_moves = []
    print("Iteracion #" + f"{iteration}")

    available_routers = [1 for _ in range(num_nodes)]
    intercambio = 1

    routers_inside = routers.copy()
    cores_inside   = cores.copy()

    while np.sum( available_routers ) > 0:
      print( "Available routers: ", available_routers )
      print( "Intercambio #" + f"{intercambio}"  )
      print( "\n" *1 )
      print( "Antes del intercambio" )
      print(  "*-" * 20 )
      print( routers_inside )
      print( cores_inside )
      print( "\n" *2 )

      print( "Costo" )
      cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores_inside, routers_inside, cost_matrix )
      print(  "*-" * 20 )
      print( "Vision Routers" )
      print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
      print(  "--" *20 )
      print(  "Vision Cores" )
      print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
      print(  "*-" * 20 )
      print( "\n" *2 )

      gain_cost_matrix = calculate_gain_cost_matrix( cores_inside, routers_inside, partition1, partition2, cost_matrix )

      print( gain_cost_matrix )

      next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, np.sum( cost_hop_matrix_routers ) ,available_routers )
      next_moves.append( next_move )


      if next_move[0] == None:
        break

      print( "Intercambio" )
      print( next_move  )
      swap = [(next_move[0],next_move[1])]

      cores_inside, routers_inside = swap_cores_between_routers(cores_inside, routers_inside, swap )


      print(  "--" * 20 )
      print( "Despues del intercambio" )
      print( routers_inside )
      print( cores_inside )
      print( "\n" *2 )

      print( "Costo" )
      cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores_inside, routers_inside, cost_matrix )
      print(  "*-" * 20 )
      print( "Vision Routers" )
      print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
      print(  "--" *20 )
      print(  "Vision Cores" )
      print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
      print(  "*-" * 20 )
      print( "\n" *2 )

      swap_routers        = update_swap_nodes( next_move )
      available_routers   = update_available_nodes( available_routers, swap_routers, num_nodes )
      intercambio += 1

    #print( next_moves )
    max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )

    print( "Intercambios"  )
    print( "*-" * 20 )
    print("Totales")
    print( next_moves )
    print( "--" * 20 )
    print( "Parciales" )
    print( partial_next_moves )
    print( "*-" * 20 )

    print( "\n" *1 )
    print( "Configuracion Inicial Iteracion" )
    print(  "*-" * 20 )
    print( routers )
    print( cores )
    print(  "*-" * 20 )
    print( "Costo" )
    cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )
    print(  "*-" * 20 )
    print( "Vision Routers" )
    print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
    print(  "--" *20 )
    print(  "Vision Cores" )
    print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
    print(  "*-" * 20 )
    print( "\n" *2 )

    cores, routers = swap_cores_between_routers( cores, routers, partial_next_moves )

    cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )


    cost_current = np.sum(cost_hop_matrix_routers)

    print( "Configuracion Final Iteracion" )
    print(  "*-" * 20 )
    print( routers )
    print( cores )
    print(  "*-" * 20 )
    print( "\n" *2 )
    print( "Costo" )
    cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )
    print(  "*-" * 20 )
    print( "Vision Routers" )
    print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
    print(  "--" *20 )
    print(  "Vision Cores" )
    print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
    print(  "*-" * 20 )
    print( "\n" *2 )

    if cost_current < cost_best:
      cost_best    = cost_current
      routers_best = routers
      cores_best   = cores
      iteration_best = iteration
      consecutive_no_change_count = 0

    else:
      consecutive_no_change_count += 1


    if consecutive_no_change_count >= 3:  # Si no hay cambios durante 3 iteraciones consecutivas
      break  # Terminar el bucle principal

  print("Best Configuration:")
  print("Iteration:", iteration_best)
  print("Cost Initial:", cost_best, " Cost Final: ", cost_initial )
  print("Routers:", routers_best)
  print("Cores:", cores_best)

  return cost_initial, iteration_best, cost_best, routers_best, cores_best



In [85]:
import random
# Optimal VOPD Cost: 4119
#routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#cores   = [9, 8, 10, 14, 7, 6, 11, 12, 0, 5, 4, 13, 1, 2, 3, 15]

#Cost: 10566
#routers = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
#cores   = [11,9,7,5,10,0,13,14,6,8,12,2,4,1,15,3]
#Cost: 10566  Cost Final:  4125
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [9, 8, 10, 14, 7, 11, 12, 13, 6, 5, 2, 1, 15, 4, 3, 0]

#Cost: 7090
#routers = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
#cores   = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
#Cost: 4217  Cost Final:  4217
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [3, 2, 1, 0, 4, 5, 6, 7, 13, 12, 14, 9, 15, 11, 10, 8]

#Cost: 4468
#routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#cores   = [0, 1, 2, 3, 7, 6, 5, 4, 8, 9, 10, 11, 12, 13, 14, 15]
#Cost: 4468  Cost Final:  4157
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [0, 1, 2, 3, 7, 6, 5, 4, 9, 12, 11, 15, 8, 13, 14, 10]





#[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#[11, 13, 5, 4, 2, 8, 12, 6, 15, 14, 10, 3, 1, 7, 0, 9]
#Cost: 9405  Cost Final:  4135
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [9, 8, 11, 12, 7, 6, 5, 13, 0, 10, 4, 14, 1, 2, 3, 15]

#[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#[8, 4, 0, 6, 14, 1, 12, 9, 2, 5, 7, 15, 13, 10, 3, 11]
#Cost: 11474  Cost Final:  4221
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [3, 4, 5, 10, 2, 15, 6, 11, 1, 12, 7, 14, 0, 13, 9, 8]

# Random
routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
cores   =  [9, 8, 10, 14, 7, 6, 11, 12, 0, 5, 4, 13, 1, 2, 3, 15]
random.shuffle(cores)
num_nodes = len(cores)

partition1 = [0, 1, 2, 3, 12, 13, 14, 15]
partition2 = [4, 5, 6, 7,  8,  9, 10, 11]

#routers = [0,1,2,3,4,5]
#cores   = [0,3,2,4,1,5]

#partition1 = [0,1,2]
#partition2 = [3,4,5]
#partition1 = [0,3,4]
#partition2 = [1,2,5]

cost_initial_kl_1, iteration_kl_1, cost_kl_1, routers_kl_1, cores_kl_1 = kernighan_lin( cores, routers, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_1)
print("Cost:", cost_initial_kl_1, " Cost Final: ", cost_kl_1)
print("Routers:", routers_kl_1)
print("Cores:", cores_kl_1)


partition1 = [0, 1, 4, 5, 8, 9, 12, 13]
partition2 = [2, 3,6,7,10,11,14,15]



cost_initial_kl_2, iteration_kl_2, cost_kl_2, routers_kl_2, cores_kl_2  = kernighan_lin( cores_kl_1, routers_kl_1, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_2)
print("Cost:", cost_initial_kl_2, " Cost Final: ", cost_kl_2)
print("Routers:", routers_kl_2)
print("Cores:", cores_kl_2)

partition1 = [0, 1, 4, 5, 10, 11, 14, 15]
partition2 = [2, 3, 6, 7,  8,  9, 12, 13]

cost_initial_kl_3, iteration_kl_3, cost_kl_3, routers_kl_3, cores_kl_3  = kernighan_lin( cores_kl_2, routers_kl_2, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_3)
print("Cost:", cost_initial_kl_3, " Cost Final: ", cost_kl_3)
print("Routers:", routers_kl_3)
print("Cores:", cores_kl_3)

partition1 = [0, 1, 2, 3, 4, 5, 6, 7]
partition2 = [8, 9,10,11,12,13,14,15]


cost_initial_kl_4, iteration_kl_4, cost_kl_4,  routers_kl_4, cores_kl_4  = kernighan_lin( cores_kl_3, routers_kl_3, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_4)
print("Cost:", cost_initial_kl_4, " Cost Final: ", cost_kl_4)
print("Routers:", routers_kl_4)
print("Cores:", cores_kl_4)


partition1 = [0, 1, 2, 3,  8,  9, 10, 11]
partition2 = [4, 5, 6, 7, 12, 13, 14, 15]
cost_initial_kl_5, iteration_kl_5, cost_kl_5,  routers_kl_5, cores_kl_5  = kernighan_lin( cores_kl_4, routers_kl_4, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_5)
print("Cost:", cost_initial_kl_5, " Cost Final: ", cost_kl_5)
print("Routers:", routers_kl_5)
print("Cores:", cores_kl_5)


partition1 = [0, 2, 4, 6, 8, 10, 12, 14]
partition2 = [1, 3, 5, 7, 9, 11, 13, 15]
cost_initial_kl_6, iteration_kl_6, cost_kl_6,  routers_kl_6, cores_kl_6  = kernighan_lin( cores_kl_5, routers_kl_5, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_6)
print("Cost:", cost_initial_kl_6, " Cost Final: ", cost_kl_6)
print("Routers:", routers_kl_6)
print("Cores:", cores_kl_6)

Streaming output truncated to the last 5000 lines.
     0.    0.   32.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   64.    0.
    48.    0.    0.    0.]
 [   0.    0.    0.    0.   81.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.]] 8628.0
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-



Available routers:  [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Intercambio #3


Antes del intercambio
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[4, 12, 11, 15, 3, 13, 5, 10, 14, 8, 7, 2, 0, 6, 1, 9]



Costo
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Vision Routers
[[  0   0   0   0   0   0 357   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0 157   0   0   0   0   0   0   0   0   0   0]
 [  0  16   0   0   0   0  16   0   0  16   0   0   0   0   0   0]
 [ 27   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [362   0   0  49   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0

In [86]:
print("Routers:", routers)
print("Cores:", cores)

print("Best Configuration 1:")
print("Iteration:", iteration_kl_1)
print("Cost:", cost_initial_kl_1, " Cost Final: ", cost_kl_1)
print("Routers:", routers_kl_1)
print("Cores:", cores_kl_1)
print("\n")

print("Best Configuration 2:")
print("Iteration:", iteration_kl_2)
print("Cost:", cost_initial_kl_2, " Cost Final: ", cost_kl_2)
print("Routers:", routers_kl_2)
print("Cores:", cores_kl_2)
print("\n")

print("Best Configuration 3:")
print("Iteration:", iteration_kl_3)
print("Cost:", cost_initial_kl_3, " Cost Final: ", cost_kl_3)
print("Routers:", routers_kl_3)
print("Cores:", cores_kl_3)
print("\n")

print("Best Configuration 4:")
print("Iteration:", iteration_kl_4)
print("Cost:", cost_initial_kl_4, " Cost Final: ", cost_kl_4)
print("Routers:", routers_kl_4)
print("Cores:", cores_kl_4)
print("\n")

print("Best Configuration 5:")
print("Iteration:", iteration_kl_5)
print("Cost:", cost_initial_kl_5, " Cost Final: ", cost_kl_5)
print("Routers:", routers_kl_5)
print("Cores:", cores_kl_5)
print("\n")

print("Best Configuration 6:")
print("Iteration:", iteration_kl_6)
print("Cost:", cost_initial_kl_6, " Cost Final: ", cost_kl_6)
print("Routers:", routers_kl_6)
print("Cores:", cores_kl_6)
print("\n")

Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [4, 12, 11, 15, 13, 3, 10, 5, 8, 14, 2, 7, 6, 0, 9, 1]
Best Configuration 1:
Iteration: 0
Cost: 3731  Cost Final:  3731
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [4, 12, 11, 15, 13, 3, 10, 5, 8, 14, 2, 7, 6, 0, 9, 1]


Best Configuration 2:
Iteration: 0
Cost: 3731  Cost Final:  3731
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [4, 12, 11, 15, 13, 3, 10, 5, 8, 14, 2, 7, 6, 0, 9, 1]


Best Configuration 3:
Iteration: 0
Cost: 3731  Cost Final:  3731
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [4, 12, 11, 15, 13, 3, 10, 5, 8, 14, 2, 7, 6, 0, 9, 1]


Best Configuration 4:
Iteration: 0
Cost: 3731  Cost Final:  3731
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [4, 12, 11, 15, 13, 3, 10, 5, 8, 14, 2, 7, 6, 0, 9, 1]


Best Configuration 5:
Iteration: 0
Cost: 3731  Cost Final:  3731
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 

## Kernighan-Lin

In [ ]:
def kernighan_lin( cores, routers, partition1, partition2, cost_matrix ):

  number_of_iteration   = 1
  number_of_interchange = 1

  for number_of_iteration in range(10):
    next_moves = []
    print( "\n"*2 )
    print( "### Iteración "+ f"{number_of_iteration}" + "###"  )

    routers_intermedio = routers.copy()
    cores_intermedio = cores.copy()

    available_routers = [1 for _ in range(num_nodes)]

    number_of_interchange = 0

    while np.sum(available_routers) :
      identation = " "*4
      level = 0
      print( identation*level +"Particiones Iniciales" )
      print(identation*level + "Routers antes del intercambio:", routers_intermedio)
      print(identation*level + "Núcleos antes del intercambio:", cores_intermedio)
      level = 1
      print(identation*level + "Particion 1" )
      print(identation*level +"Routers antes del intercambio:", [routers_intermedio[i] for i in partition1] )
      print(identation*level +"Núcleos antes del intercambio:", [cores_intermedio[i] for i in partition1])
      print(identation*level +"Particion 2" )
      print(identation*level +"Routers antes del intercambio:", [routers_intermedio[i] for i in partition2])
      print(identation*level +"Núcleos antes del intercambio:", [cores_intermedio[i] for i in partition2])

      cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores_intermedio, routers_intermedio, cost_matrix )

      print( identation*level + "*-" * 20 )
      print( identation*level +"Vision Routers" )
      print( identation*level, cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
      print( identation*level + "--" *20 )
      print(identation*level + "Vision Cores" )
      print( identation*level , cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
      print( identation*level + "*-" * 20 )
      print( "\n" *2 )

      level = 2
      print( identation*level + "# Intercambio #" + f"{number_of_interchange}" )


      difference_cost_vector =  calculate_difference_cost_vector( cost_hop_matrix_routers, partition1, partition2, num_nodes  )
      gain_cost_matrix       =  calculate_gain_cost_matrix( difference_cost_vector, partition1, partition2, None, cost_matrix )
      next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_routers )



      if next_move[0] == None:
        break

      swap =[(next_move[0],next_move[1])]

      cores_intermedio, routers_intermedio = swap_cores_between_routers(cores_intermedio, routers_intermedio, swap )

      swap_routers        = update_swap_nodes( next_move )
      available_routers   = update_available_nodes( available_routers, swap_routers )
      next_moves.append( next_move )

      level = 0
      print( identation*level +"Particiones Intermedias" )
      print(identation*level + "Routers despues del intercambio:", routers_intermedio)
      print(identation*level + "Núcleos despues del intercambio:", cores_intermedio)
      level = 1
      print(identation*level + "Particion 1" )
      print(identation*level +"Routers despues del intercambio:", [routers_intermedio[i] for i in partition1] )
      print(identation*level +"Núcleos despues del intercambio:", [cores_intermedio[i] for i in partition1])
      print(identation*level +"Particion 2" )
      print(identation*level +"Routers despues del intercambio:", [routers_intermedio[i] for i in partition2])
      print(identation*level +"Núcleos despues del intercambio:", [cores_intermedio[i] for i in partition2])


      cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores_intermedio, routers_intermedio, cost_matrix )

      print( identation*level + "*-" * 20 )
      print( identation*level +"Vision Routers" )
      print( identation*level , cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
      print( identation*level + "--" *20 )
      print(identation*level + "Vision Cores" )
      print( identation*level , cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
      print( identation*level + "*-" * 20 )
      print( "\n" *2 )





      number_of_interchange += 1

      #'''
      print("-"*40)
      print("Difference cost Vector")
      print( difference_cost_vector )
      print("-"*40)

      print("\n")

      print("-"*40)
      print("Gain Cost Matrix")
      print( gain_cost_matrix )
      print("-"*40)

      print("\n")

      print("-"*40)
      print("Particiones Potenciales")
      print( "Movimiento Potencial:", next_move )
      print( "Partition1_potencial:", partition1 )
      print( "Partition2_potencial:", partition2 )
      #'''


    max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )

    print(max_partial_sum, partial_next_moves)

    print( "Movimientos Potenciales: ", next_moves )





    print( "Movimientos Finales:", partial_next_moves )


    if max_partial_sum <= 0:
      print("Terminando debido a que max_partial_sum es igual o menor a 0")
      level = 1
      print(identation*level + "Routers antes del intercambio:", routers)
      print(identation*level + "Núcleos antes del intercambio:", cores)
      level = 2
      print( identation*level + "Particion 1" )
      print(identation*level +"Routers antes del intercambio:", [routers[i] for i in partition1] )
      print(identation*level +"Núcleos antes del intercambio:", [cores[i] for i in partition1])
      print(identation*level +"Particion 2" )
      print(identation*level +"Routers antes del intercambio:", [routers[i] for i in partition2])
      print(identation*level +"Núcleos antes del intercambio:", [cores[i] for i in partition2])

      return partition1, partition2


    partition1_post, partition2_post = swap_cores( partial_next_moves, partition1, partition2 )

    level = 0
    print( identation*level +"Particiones Iniciales" )
    print(identation*level + "Routers antes del intercambio:", routers)
    print(identation*level + "Núcleos antes del intercambio:", cores)
    cores, routers = swap_cores_between_routers(cores, routers, partial_next_moves )
    print( identation*level +"Particiones Finales" )
    print(identation*level + "Routers antes del intercambio:", routers)
    print(identation*level + "Núcleos antes del intercambio:", cores)

    cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

    print( "*-" * 20 )
    print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
    print( "--" *20 )
    print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
    print( "*-" * 20 )
    print( "\n" *2 )





    if partition1_post == partition2 and partition2_post == partition1:
      print( "Particiones Finales Iteracion" )
      print( partition1, partition2_post )
      print( partition2, partition1_post )
      return partition1, partition2






In [ ]:

routers = [0,1,2,3,4,5]
cores   = [2,1,5,3,4,0]

kernighan_lin( cores, routers, partition1, partition2, cost_matrix )

print(  "\n" )
print( "Particiones Iniciales" + "\t"*2 + "Particiones Finales" )
#print(  f"{partition1}" + "\t"*1 + f"{partition1_kl}" )
#print(  f"{partition2}" + "\t"*1 + f"{partition2_kl}" )




### Iteración 0###
Particiones Iniciales
Routers antes del intercambio: [0, 1, 2, 3, 4, 5]
Núcleos antes del intercambio: [2, 1, 5, 3, 4, 0]
    Particion 1
    Routers antes del intercambio: [0, 1, 2]
    Núcleos antes del intercambio: [2, 1, 5]
    Particion 2
    Routers antes del intercambio: [3, 4, 5]
    Núcleos antes del intercambio: [3, 4, 0]


ValueError: ignored

# Version #*3* Pulida

## Funciones

In [77]:
import numpy as np
import math

def create_external_adj_matrix(partition1, partition2, num_nodes):
  external_adj_matrix = np.zeros((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      external_adj_matrix[node1][node2] = 1
      external_adj_matrix[node2][node1] = 1

  return external_adj_matrix


def create_internal_adj_matrix( partition1, partition2, num_nodes ):

  internal_adj_matrix = np.ones((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      internal_adj_matrix[node1][node2] = 0
      internal_adj_matrix[node2][node1] = 0

  return internal_adj_matrix

def create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes ):
  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
  external_cost_matrix = np.zeros((num_nodes,num_nodes))

  for i in range(num_nodes):
    for j in range(num_nodes):
      external_cost_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

  return external_cost_matrix


def create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes ):

  internal_adj_matrix = create_internal_adj_matrix(partition1, partition2, num_nodes)
  internal_cost_matrix = np.zeros((num_nodes,num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
      if i != j:
        internal_cost_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

  return internal_cost_matrix


def calculate_difference_cost_vector( cost_matrix, partition1, partition2, num_nodes ):

  external_cost_matrix = create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes )


  difference_cost_matrix = np.zeros((num_nodes, num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
     difference_cost_matrix[i][j] += external_cost_matrix[j][i] - internal_cost_matrix[j][i]

  #
  '''
  print("*-"*20)
  print( "External Cost Matrix" )
  print( external_cost_matrix )


  print("-"*40)
  print( "Internal Cost Matrix" )
  print( internal_cost_matrix )

  print("*-"*20)
  print( "\n" )
  #'''
  #return np.dot([1,1,1,1,1,1],difference_cost_matrix)
  #return np.dot(np.array(num_nodes), difference_cost_matrix)
  return np.dot([1 for _ in range(num_nodes)], difference_cost_matrix)

def update_difference_cost_vector( cost_matrix, difference_cost_vector, partition1, partition2, available_nodes, swap_nodes  ):

    internal_adj_matrix = create_internal_adj_matrix( partition1, partition2, num_nodes)
    difference_cost_internal_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_internal_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

    external_adj_matrix = create_external_adj_matrix( partition1, partition2, num_nodes)
    difference_cost_external_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
          difference_cost_external_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

    difference_cost_prime_matrix = np.zeros((num_nodes,num_nodes))
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_prime_matrix[i][j] = difference_cost_internal_matrix[i][j] - difference_cost_external_matrix[i][j]

    difference_cost_vector          = np.multiply(available_nodes, difference_cost_vector)
    difference_cost_internal_vector = np.dot(swap_nodes, difference_cost_internal_matrix)
    difference_cost_external_vector = np.dot(swap_nodes, difference_cost_external_matrix)
    difference_cost_prime_vector    = difference_cost_vector + 2 * difference_cost_internal_vector - 2 * difference_cost_external_vector

    #'''
    print(internal_adj_matrix)
    print("*-"*20)
    print( "Difference Cost Internal Matrix")
    for row in difference_cost_internal_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost Internal Vector")
    print( difference_cost_internal_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost External Matrix")
    for row in difference_cost_external_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost External Vector")
    print( difference_cost_external_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost Vector")
    print( difference_cost_vector)

    print("-"*40)

    print( "Difference Cost Updated Vector")
    print( difference_cost_prime_vector )

    print("*-"*20)
    print("\n")
    #'''
    return difference_cost_prime_vector

def calculate_gain_cost_matrix( difference_cost_vector, partition1, partition2 ,external_adj_matrix, cost_matrix ):
  gain_cost_matrix = np.full((num_nodes,num_nodes), 0)

  external_cost_matrix = create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)

  for i in range(num_nodes):
    for j in range(num_nodes):
      if external_adj_matrix[i][j] == 1:
        gain_cost_matrix[i][j] = difference_cost_vector[i] + difference_cost_vector[j] - 2 * cost_matrix[i][j]

  return gain_cost_matrix


def swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes ):
  partition1_copy = set(partition1)
  partition2_copy = set(partition2)
  max_gain = float('inf')
  max_i, max_j = None, None

  for i in range( num_nodes ):
    for j in range( num_nodes ):
      if gain_cost_matrix[i][j] < max_gain  and gain_cost_matrix[i][j] != 0 and available_nodes[i] == 1 and available_nodes[j] == 1 and i in partition1_copy and j in partition2_copy:
        max_gain = gain_cost_matrix[i][j]
        max_i = i
        max_j = j

  #print( max_i, max_j, max_gain )
  next_move = [ max_i, max_j, max_gain]

  for ci in partition1_copy:
    for cj in partition2_copy:
      if max_i== ci and max_j == cj:
        partition1_copy.remove(max_i)
        partition2_copy.remove(max_j)
        partition1_copy.add(max_j)
        partition2_copy.add(max_i)
        #print("Hubo intercambio Parcial!!!!!", (max_i, max_j))

  return next_move

def max_subarray_sum(next_moves):
    select_next_moves = []
    current_sum = 0
    max_sum = np.inf

    for next_move in next_moves:
      if math.isfinite(next_move[2]):
        current_sum = int(next_move[2])
        #print( next_move, current_sum )
        if current_sum < max_sum:
          max_sum = current_sum
          select_next_moves.append( next_move )

    return max_sum, select_next_moves

def swap_cores(  partial_next_moves, partition1, partition2  ):
  for next_move in partial_next_moves:
    for ci in partition1:
     for cj in partition2:
        if ci == next_move[0] and cj == next_move[1]:
          partition1.remove(ci)
          partition2.remove(cj)
          partition1.add(cj)
          partition2.add(ci)
  return partition1, partition2

def swap_cores(partial_next_moves, partition1, partition2):
    new_partition1 = set(partition1)
    new_partition2 = set(partition2)

    for next_move in partial_next_moves:
        node_from = next_move[0]
        node_to = next_move[1]

        if node_from in new_partition1 and node_to in new_partition2:
            new_partition1.remove(node_from)
            new_partition2.remove(node_to)
            new_partition1.add(node_to)
            new_partition2.add(node_from)
            #print( "Hubo intercambio Final!!!!!" + f"{node_from,node_to}"  )

    return new_partition1, new_partition2

def update_available_nodes( available_nodes, swap_nodes, num_nodes ):
  return  [a ^ b for a, b in zip(available_nodes, swap_nodes)]

def update_swap_nodes( next_move ):
  indices = next_move[0:2]
  swap_nodes = [1 if i in indices else 0 for i in range(num_nodes)]
  return swap_nodes


## Topology routers

In [78]:
import numpy as np

def create_hop_matrix_routers( adj_matrix_routers ):
  hop_matrix_routers = adj_matrix_routers.astype(float)
  hop_matrix_routers[hop_matrix_routers == 0] = np.inf

  for intermediate_router in range( adj_matrix_routers.shape[1] ):
    for source_router in range( adj_matrix_routers.shape[0] ):
      for destination_router in range( adj_matrix_routers.shape[1] ):
        if source_router == destination_router:
          hop_matrix_routers[source_router, destination_router] = 0
        else:
          hop_matrix_routers[source_router, destination_router]  = min(
              hop_matrix_routers[source_router, destination_router],
              hop_matrix_routers[source_router, intermediate_router] + hop_matrix_routers[intermediate_router, destination_router]
          )
  hop_matrix_routers = hop_matrix_routers.astype(int)

  return hop_matrix_routers

def create_adj_matrix_cores_to_routers( cores, routers ):
    adj_matrix_cores = np.zeros( (len(cores), len(routers)), dtype=int )

    for index in range(len(cores)) :
      adj_matrix_cores[cores[index], routers[index]] = 1

    return adj_matrix_cores

def create_hop_matrix_cores_tdg( tdg_matrix, hop_matrix_cores ):
  hop_matrix_cores_tdg = hop_matrix_cores.astype(int)

  for source_core_index in range(hop_matrix_cores.shape[0]):
    for destination_core_index in range(hop_matrix_cores.shape[1]):
      if tdg_matrix[source_core_index,destination_core_index] > 0:
        hop_matrix_cores_tdg[source_core_index,destination_core_index] = hop_matrix_cores[source_core_index,destination_core_index]
      else:
        hop_matrix_cores_tdg[source_core_index,destination_core_index] = 0

  return hop_matrix_cores_tdg

In [79]:
def create_hop_matrix_cores( hop_matrix_routers, adj_matrix_cores_to_routers ):
  hop_matrix_cores = np.zeros_like( adj_matrix_cores_to_routers )

  for source_core in range( adj_matrix_cores_to_routers.shape[0] ):
    for destination_core in range( adj_matrix_cores_to_routers.shape[1] ):
      if source_core == destination_core:
        hop_matrix_cores[source_core,destination_core] = 0
      else:
        source_router = list(adj_matrix_cores_to_routers[source_core]).index(1)
        destination_router = list(adj_matrix_cores_to_routers[destination_core]).index(1)
        hop_matrix_cores[source_core,destination_core] = hop_matrix_routers[source_router,destination_router]

  return hop_matrix_cores

def create_cost_routers_matrix( cost_cores_matrix, adj_matrix_cores_to_routers ):
  cost_routers_matrix = np.zeros_like( adj_matrix_cores_to_routers )

  for source_router in range( adj_matrix_cores_to_routers.shape[0] ):
    for destination_router in range( adj_matrix_cores_to_routers.shape[1] ):
      if source_router == destination_router:
        cost_routers_matrix[source_router,destination_router] = 0
      else:
        source_core = list(adj_matrix_cores_to_routers[:,source_router]).index(1)
        destination_core = list(adj_matrix_cores_to_routers[:,destination_router]).index(1)
        cost_routers_matrix[source_router,destination_router] = cost_cores_matrix[source_core,destination_core]

  return cost_routers_matrix



def create_cost_routers_matrix( cost_cores_matrix, adj_matrix_cores_to_routers ):
  cost_routers_matrix = np.zeros_like( adj_matrix_cores_to_routers )

  for source_router in range( adj_matrix_cores_to_routers.shape[0] ):
    for destination_router in range( adj_matrix_cores_to_routers.shape[1] ):
      if source_router == destination_router:
        cost_routers_matrix[source_router,destination_router] = 0
      else:
        source_core = list(adj_matrix_cores_to_routers[:,source_router]).index(1)
        destination_core = list(adj_matrix_cores_to_routers[:,destination_router]).index(1)
        cost_routers_matrix[source_router,destination_router] = cost_cores_matrix[source_core,destination_core]

  return cost_routers_matrix



def create_cost_matrix( cores, routers, cost_cores_matrix ):
  adj_matrix_routers = np.array(
                      [[0, 1, 0, 1, 0, 0],
                      [1, 0, 1, 0, 1, 0],
                      [0, 1, 0, 0, 0, 1],
                      [1, 0, 0, 0, 1, 0],
                      [0, 1, 0, 1, 0, 1],
                      [0, 0, 1, 0, 1, 0]]
                      )

  adj_matrix_routers = np.array(
                     [[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                      [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                      [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                      [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
                      [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                      [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
                      [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0],
                      [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
                      [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]]
                      )


  hop_matrix_routers          = create_hop_matrix_routers( adj_matrix_routers )
  adj_matrix_cores_to_routers = create_adj_matrix_cores_to_routers( cores, routers )


  # Cores Vision
  #hop_matrix_cores            = create_hop_matrix_cores( hop_matrix_routers, adj_matrix_cores_to_routers )
  #cost_hop_matrix_cores       = cost_cores_matrix * hop_matrix_cores

  inicio = time.time()
  # Routers Vision
  cost_routers_matrix        = create_cost_routers_matrix( cost_cores_matrix, adj_matrix_cores_to_routers )
  fin = time.time()
  #print( "Etapa 1: ", fin - inicio )


  inicio = time.time()
  cost_hop_matrix_routers    = cost_routers_matrix * hop_matrix_routers
  fin = time.time()
  #print( "Etapa 2: ", fin - inicio )

  #
  '''
  print( "*-" * 20 )
  print( adj_matrix_routers )
  print( adj_matrix_cores_to_routers )
  print( "--" *20 )
  print( hop_matrix_routers )
  print( hop_matrix_cores )
  print( "--" *20 )
  print( cost_hop_matrix_routers )
  print( cost_hop_matrix_cores )
  print( "*-" * 20 )
  print( "\n" *2 )
  #'''

  return  cost_hop_matrix_routers


def swap_cores_between_routers( cores, routers, swaps ):
  for swap in swaps:
    router1 = swap[0]
    router2 = swap[1]
    # Verificar si los routers son válidos
    if router1 < 0 or router1 >= len(routers) or router2 < 0 or router2 >= len(routers):
      print("Los números de router no son válidos.")
      continue

    # Intercambiar los núcleos entre los routers
    temporal_core = cores[router1]
    cores[router1] = cores[router2]
    cores[router2] = temporal_core

  return cores, routers


def swap_cores_between_routers( cores, routers, swaps ):
  new_cores = cores.copy()  # Crear una copia de los núcleos originales
  new_routers = routers.copy()  # Crear una copia de los enrutadores originales

  for swap in swaps:
    router1 = swap[0]
    router2 = swap[1]
    # Verificar si los routers son válidos
    if router1 < 0 or router1 >= len(new_routers) or router2 < 0 or router2 >= len(new_routers):
      print("Los números de router no son válidos.")
      continue

    # Intercambiar los núcleos entre los routers en las copias
    temporal_core = new_cores[router1]
    new_cores[router1] = new_cores[router2]
    new_cores[router2] = temporal_core

  return new_cores, new_routers


## Ejemplo


In [68]:
## DSP
cost_matrix = np.array(
              [[0, 200,   0,    0,   0,   0],
               [0,   0, 200,  600,   0,   0],
               [0,   0,   0,    0,   0,   0],
               [0, 600,   0,    0, 200, 200],
               [0,   0,   0,  200,   0,   0],
               [0,   0,   0,  200,   0,   0]]
              )

# Particiones Iniciales		Particiones Finales
#    [0, 1, 2]	               {1, 3, 5}
#    [3, 4, 5]	               {0, 2, 4}


# Particiones Iniciales
#    [0, 1, 2]
#    [3, 4, 5]

cores   = [2,1,5,3,4,0]
routers = [0,1,2,3,4,5]

print("Núcleos antes del intercambio:", cores)
print("Routers antes del intercambio:", routers)

num_nodes = len(cores)

partition1 = [0,1,2]
partition2 = [3,4,5]



cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

print( "*-" * 20 )
print( "Vision Cores" )
print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
print( "--" *20 )
print( "Vision routers" )
print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
print( "*-" * 20 )
print( "\n" *2 )




# Ejemplo de uso

swaps =[(2,4)]
cores, routers = swap_cores_between_routers(cores, routers, swaps )


swaps =[(0,3)]
cores, routers = swap_cores_between_routers(cores, routers, swaps )


print("Núcleos después del intercambio:", cores)
print("Routers después del intercambio:", routers)



# Particiones Finales
#    {1, 3, 5}
#    {0, 2, 4}

partition1 = [0,1,2]
partition2 = [3,4,5]

cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

print( "*-" * 20 )
print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
print( "--" *20 )
print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
print( "*-" * 20 )
print( "\n" *2 )



Núcleos antes del intercambio: [2, 1, 5, 3, 4, 0]
Routers antes del intercambio: [0, 1, 2, 3, 4, 5]


ValueError: ignored

In [80]:

routers = [0,1,2,3,4,5]
cores   = [2,1,5,3,4,0]

routers = [0,1,2,3,4,5]
cores   = [2,5,1,0,4,3]

num_nodes = len(cores)

partition1 = [0,1,2]
partition2 = [3,4,5]

cost_matrix = np.array(
              [[0, 200,   0,    0,   0,   0],
               [0,   0, 200,  600,   0,   0],
               [0,   0,   0,    0,   0,   0],
               [0, 600,   0,    0, 200, 200],
               [0,   0,   0,  200,   0,   0],
               [0,   0,   0,  200,   0,   0]]
              )


#VOPD
#Cost: 7090
routers = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
cores   = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
num_nodes = len(cores)

partition1 = [0, 1, 2, 3, 4, 5, 6, 7]
partition2 = [8, 9,10,11,12,13,14,15]

#VOPD Best
#5127
#'''
#Cost: 5127
#routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#cores   = [0, 1, 2, 3, 10, 8, 9, 15, 14, 11, 7, 4, 12, 13, 5, 6]
#'''

# Optimal VOPD
#routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#cores   = [9, 8, 10, 14, 7, 6, 11, 12, 0, 5, 4, 13, 1, 2, 3, 15]

#VOPD Best
#Cost: 4468
routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
cores   = [0, 1, 2, 3, 7, 6, 5, 4, 8, 9, 10, 11, 12, 13, 14, 15]

#partition1 = [0, 1, 4, 5, 8, 9, 12, 13]
#partition2 = [2, 3,6,7,10,11,14,15]

#Cost: 10566
routers = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
cores   = [11,9,7,5,10,0,13,14,6,8,12,2,4,1,15,3]
num_nodes = len(cores)




cost_matrix = np.array(
   [[  0.,  70.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0., 362.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,  49.],
    [  0.,   0.,   0.,   0.,   0., 357.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0., 353.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0., 300.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 313., 500.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  94.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 313.,   0.,   0.,   0.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  16.,   0.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,  16.,   0.,   0.,  16.,   0.,   0.,   0.,  16.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 157.,  0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 16.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  16.,   0.,  16.,   0.,  0.,   0.],
    [  0.,   0.,   0.,   0.,  27.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  0.,   0.] ]
)



In [113]:
import time

def calculate_gain_cost_matrix( cores, routers, partition1, partition2, cost_matrix ):


  gain_cost_matrix = np.full((num_nodes,num_nodes), 0)

  external_cost_matrix = create_external_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, partition1, partition2, num_nodes )
  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)



  for i in range(num_nodes):
    for j in range(num_nodes):
      if external_adj_matrix[i][j] == 1:

        swap =[(i,j)]
        #print("Intercambio: ", swap)
        #print(  "*-" * 20 )
        #print( "Configuracion Inicial" )
        #print( routers )
        #print( cores )

        cores_swap, routers_swap = swap_cores_between_routers(cores, routers, swap )
        #print( "Configuracion Final" )
        #print( routers_swap )
        #print( cores_swap )
        #print(  "*-" * 20 )

        cost_hop_matrix_routers = create_cost_matrix( cores_swap, routers_swap, cost_matrix )
        #print(  "*-" * 20 )
        #print( "Vision Routers" )
        #print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
        #print(  "--" *20 )
        #print(  "Vision Cores" )
        #print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
        #print(  "*-" * 20 )
        #print( "\n" *2 )

        difference_cost_vector =  calculate_difference_cost_vector( cost_hop_matrix_routers, partition1, partition2, num_nodes  )
        #print( "Diffenrence Vector" )
        #print( difference_cost_vector )
        gain_cost_matrix[i][j] = np.sum( cost_hop_matrix_routers ) #difference_cost_vector[i] + difference_cost_vector[j] - 2 * cost_matrix[i][j]
        #print( "Gain Cost" )
        #print( gain_cost_matrix[i][j]  )
        #print( "\n" *2 )


  return gain_cost_matrix


def kernighan_lin( cores, routers, partition1, partition2, num_nodes ):
  print( "Kernighan-Lin Algorithm" )

  cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

  '''
  print(  "*-" * 20 )
  print( "Vision Routers" )
  print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
  print(  "--" *20 )
  print(  "Vision Cores" )
  print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
  print(  "*-" * 20 )
  print( "\n" *2 )
  '''
  external_cost_matrix = create_external_cost_matrix( cost_hop_matrix_routers, partition1, partition2, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_hop_matrix_routers, partition1, partition2, num_nodes )

  '''
  print("*-"*20)
  print( "External Cost Matrix" )
  print( external_cost_matrix )


  print("--"*20)
  print( "Internal Cost Matrix" )
  print( internal_cost_matrix )
  print("*-"*20)
  '''

  routers_initial = routers
  cores_initial = cores


  cost_initial =  np.sum( cost_hop_matrix_routers )
  cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

  cost_best       = np.sum( cost_hop_matrix_routers )
  routers_best    = routers
  cores_best      = cores

  # Después de definir cost_best, routers_best y cores_best al comienzo del código
  consecutive_no_change_count = 0  # Inicializamos el contador en cero
  iteration_best = 0
  for iteration in range(1,11):

    next_moves = []
    print("Iteracion #" + f"{iteration}", end = " " *4 )

    available_routers = [1 for _ in range(num_nodes)]
    intercambio = 1

    routers_inside = routers.copy()
    cores_inside   = cores.copy()

    cost_hop_matrix_routers = create_cost_matrix( cores_best, routers_best, cost_matrix )
    print("Cost Best: ", np.sum(cost_hop_matrix_routers) , end = " "*4 )

    cost_hop_matrix_routers = create_cost_matrix( cores_inside, routers_inside, cost_matrix )
    print( "Cost Initial: ", np.sum( cost_hop_matrix_routers ))

    while np.sum( available_routers ) > 0:

      print( "  " + "Available routers: ", available_routers )
      print( "  " + "Intercambio #" + f"{intercambio}", end=" "* 4  )



      gain_cost_matrix = calculate_gain_cost_matrix( cores_inside, routers_inside, partition1, partition2, cost_matrix )
      #print( gain_cost_matrix )


      next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_routers )
      next_moves.append( next_move )

      #print( "\n" *1 )
      #print( "Antes del intercambio" )
      #print(  "*-" * 20 )
      #print( routers_inside )
      #print( cores_inside )



      #print( next_move )
      if next_move[0] == None:
        break
      swap = [(next_move[0],next_move[1])]

      cores_inside, routers_inside = swap_cores_between_routers(cores_inside, routers_inside, swap )


      #print(  "--" * 20 )
      #print( "Despues del intercambio" )
      #print( routers_inside )
      #print( cores_inside )
      #print( "\n" *2


      #print( "Costo" )
      cost_hop_matrix_routers = create_cost_matrix( cores_inside, routers_inside, cost_matrix )
      print( "Cost Current: ", np.sum(cost_hop_matrix_routers)  )

      #print(  "*-" * 20 )
      #print( "Vision Routers" )
      #print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
      #print(  "--" *20 )
      #print(  "Vision Cores" )
      #print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
      #print(  "*-" * 20 )
      #print( "\n" *2 )


      swap_routers        = update_swap_nodes( next_move )
      available_routers   = update_available_nodes( available_routers, swap_routers, num_nodes )
      intercambio += 1


    #print( next_moves )
    max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )
    #print( partial_next_moves )

    #print( "\n" *1 )
    #print( "Configuracion Inicial Iteracion" )
    #print(  "*-" * 20 )
    #print( routers )
    #print( cores )
    #print(  "*-" * 20 )

    cores, routers = swap_cores_between_routers( cores, routers, partial_next_moves )

    cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )


    cost_current = np.sum(cost_hop_matrix_routers)

    #print( "Configuracion Final Iteracion" )
    #print(  "*-" * 20 )
    #print( routers )
    #print( cores )
    #print(  "*-" * 20 )
    #print( "\n" *2 )

    if cost_current < cost_best:
      cost_best    = cost_current
      routers_best = routers
      cores_best   = cores
      iteration_best = iteration
      consecutive_no_change_count = 0

    else:
      consecutive_no_change_count += 1


    if consecutive_no_change_count >= 3:  # Si no hay cambios durante 3 iteraciones consecutivas
      break  # Terminar el bucle principal
    print("\n")

  print("Results Kernighan-Lin")
  print(" "*4 + "Solution Optimal Iteration:", iteration_best, end =" "*4 )
  print(" "*4 + "Configuration Initial" + " "*4 +"Cost: ", cost_initial )
  print(" "*4 + "Routers:", routers_initial)
  print(" "*4 + "Cores:  ", cores_initial)
  print(" "*4 + "Configuration Best" + " "*4 + "Cost: ", cost_best )
  print(" "*4 + "Routers:", routers_best)
  print(" "*4 + "Cores:  ", cores_best)

  print("\n" )

  return cost_initial, iteration_best, cost_best, routers_best, cores_best



In [114]:
import random
# Optimal VOPD Cost: 4119
routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
cores   = [9, 8, 10, 14, 7, 6, 11, 12, 0, 5, 4, 13, 1, 2, 3, 15]

#Cost: 10566
routers = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
cores   = [11,9,7,5,10,0,13,14,6,8,12,2,4,1,15,3]
#Cost: 10566  Cost Final:  4125
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [9, 8, 10, 14, 7, 11, 12, 13, 6, 5, 2, 1, 15, 4, 3, 0]

#Cost: 7090
routers = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
cores   = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
#Cost: 4217  Cost Final:  4217
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [3, 2, 1, 0, 4, 5, 6, 7, 13, 12, 14, 9, 15, 11, 10, 8]

#Cost: 4468
routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
cores   = [0, 1, 2, 3, 7, 6, 5, 4, 8, 9, 10, 11, 12, 13, 14, 15]
#Cost: 4468  Cost Final:  4157
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [0, 1, 2, 3, 7, 6, 5, 4, 9, 12, 11, 15, 8, 13, 14, 10]



# Random
#
'''
routers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
cores   = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
random.shuffle(cores)
#'''

#[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#[11, 13, 5, 4, 2, 8, 12, 6, 15, 14, 10, 3, 1, 7, 0, 9]
#Cost: 9405  Cost Final:  4135
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [9, 8, 11, 12, 7, 6, 5, 13, 0, 10, 4, 14, 1, 2, 3, 15]

routers =  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
cores    = [8, 4, 0, 6, 14, 1, 12, 9, 2, 5, 7, 15, 13, 10, 3, 11]
#Cost: 11474  Cost Final:  4221
#Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#Cores: [3, 4, 5, 10, 2, 15, 6, 11, 1, 12, 7, 14, 0, 13, 9, 8]

num_nodes = len(cores)

partition1 = [0, 1, 2, 3, 12, 13, 14, 15]
partition2 = [4, 5, 6, 7,  8,  9, 10, 11]

cost_initial_kl_1, iteration_kl_1, cost_kl_1, routers_kl_1, cores_kl_1 = kernighan_lin( cores, routers, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_1)
print("Cost:", cost_initial_kl_1, " Cost Final: ", cost_kl_1)
print("Routers:", routers_kl_1)
print("Cores:", cores_kl_1)


partition1 = [0, 1, 4, 5, 8, 9, 12, 13]
partition2 = [2, 3,6,7,10,11,14,15]

cost_initial_kl_2, iteration_kl_2, cost_kl_2, routers_kl_2, cores_kl_2  = kernighan_lin( cores_kl_1, routers_kl_1, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_2)
print("Cost:", cost_initial_kl_2, " Cost Final: ", cost_kl_2)
print("Routers:", routers_kl_2)
print("Cores:", cores_kl_2)

partition1 = [0, 1, 4, 5, 10, 11, 14, 15]
partition2 = [2, 3, 6, 7,  8,  9, 12, 13]

cost_initial_kl_3, iteration_kl_3, cost_kl_3, routers_kl_3, cores_kl_3  = kernighan_lin( cores_kl_2, routers_kl_2, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_3)
print("Cost:", cost_initial_kl_3, " Cost Final: ", cost_kl_3)
print("Routers:", routers_kl_3)
print("Cores:", cores_kl_3)

partition1 = [0, 1, 2, 3, 4, 5, 6, 7]
partition2 = [8, 9,10,11,12,13,14,15]


cost_initial_kl_4, iteration_kl_4, cost_kl_4,  routers_kl_4, cores_kl_4  = kernighan_lin( cores_kl_3, routers_kl_3, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_4)
print("Cost:", cost_initial_kl_4, " Cost Final: ", cost_kl_4)
print("Routers:", routers_kl_4)
print("Cores:", cores_kl_4)


partition1 = [0, 1, 2, 3,  8,  9, 10, 11]
partition2 = [4, 5, 6, 7, 12, 13, 14, 15]
cost_initial_kl_5, iteration_kl_5, cost_kl_5,  routers_kl_5, cores_kl_5  = kernighan_lin( cores_kl_4, routers_kl_4, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_5)
print("Cost:", cost_initial_kl_5, " Cost Final: ", cost_kl_5)
print("Routers:", routers_kl_5)
print("Cores:", cores_kl_5)


partition1 = [0, 2, 4, 6, 8, 10, 12, 14]
partition2 = [1, 3, 5, 7, 9, 11, 13, 15]
cost_initial_kl_6, iteration_kl_6, cost_kl_6,  routers_kl_6, cores_kl_6  = kernighan_lin( cores_kl_5, routers_kl_5, partition1, partition2, num_nodes )

print("Best Configuration:")
print("Iteration:", iteration_kl_6)
print("Cost:", cost_initial_kl_6, " Cost Final: ", cost_kl_6)
print("Routers:", routers_kl_6)
print("Cores:", cores_kl_6)

Kernighan-Lin Algorithm
Iteracion #1    Cost Best:  11474    Cost Initial:  11474
  Available routers:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  Intercambio #1    Cost Current:  9141
  Available routers:  [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  Intercambio #2    Cost Current:  7671
  Available routers:  [0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1]
  Intercambio #3    Cost Current:  6353
  Available routers:  [0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1]
  Intercambio #4    Cost Current:  5920
  Available routers:  [0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1]
  Intercambio #5    Cost Current:  5844
  Available routers:  [0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1]
  Intercambio #6    Cost Current:  6010
  Available routers:  [0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1]
  Intercambio #7    Cost Current:  7663
  Available routers:  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
  Intercambio #8    Cost Current:  9803


Iteracion #2    Cost Best:  

In [105]:
print("Routers:", routers)
print("Cores:", cores)

print("Best Configuration 1:")
print("Iteration:", iteration_kl_1)
print("Cost:", cost_initial_kl_1, " Cost Final: ", cost_kl_1)
print("Routers:", routers_kl_1)
print("Cores:", cores_kl_1)
print("\n")

print("Best Configuration 2:")
print("Iteration:", iteration_kl_2)
print("Cost:", cost_initial_kl_2, " Cost Final: ", cost_kl_2)
print("Routers:", routers_kl_2)
print("Cores:", cores_kl_2)
print("\n")

print("Best Configuration 3:")
print("Iteration:", iteration_kl_3)
print("Cost:", cost_initial_kl_3, " Cost Final: ", cost_kl_3)
print("Routers:", routers_kl_3)
print("Cores:", cores_kl_3)
print("\n")

print("Best Configuration 4:")
print("Iteration:", iteration_kl_4)
print("Cost:", cost_initial_kl_4, " Cost Final: ", cost_kl_4)
print("Routers:", routers_kl_4)
print("Cores:", cores_kl_4)
print("\n")

print("Best Configuration 5:")
print("Iteration:", iteration_kl_5)
print("Cost:", cost_initial_kl_5, " Cost Final: ", cost_kl_5)
print("Routers:", routers_kl_5)
print("Cores:", cores_kl_5)
print("\n")

print("Best Configuration 6:")
print("Iteration:", iteration_kl_6)
print("Cost:", cost_initial_kl_6, " Cost Final: ", cost_kl_6)
print("Routers:", routers_kl_6)
print("Cores:", cores_kl_6)
print("\n")

Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [8, 4, 0, 6, 14, 1, 12, 9, 2, 5, 7, 15, 13, 10, 3, 11]
Best Configuration 1:
Iteration: 4
Cost: 11474  Cost Final:  4669
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [3, 4, 5, 10, 0, 15, 6, 14, 2, 12, 7, 11, 1, 13, 9, 8]


Best Configuration 2:
Iteration: 0
Cost: 4669  Cost Final:  4669
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [3, 4, 5, 10, 0, 15, 6, 14, 2, 12, 7, 11, 1, 13, 9, 8]


Best Configuration 3:
Iteration: 3
Cost: 4669  Cost Final:  4221
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [3, 4, 5, 10, 2, 15, 6, 11, 1, 12, 7, 14, 0, 13, 9, 8]


Best Configuration 4:
Iteration: 0
Cost: 4221  Cost Final:  4221
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Cores: [3, 4, 5, 10, 2, 15, 6, 11, 1, 12, 7, 14, 0, 13, 9, 8]


Best Configuration 5:
Iteration: 0
Cost: 4221  Cost Final:  4221
Routers: [0, 1, 2, 3, 4, 5, 6, 7, 8,

## Kernighan-Lin

In [65]:
def kernighan_lin( cores, routers, partition1, partition2, cost_matrix ):

  number_of_iteration   = 1
  number_of_interchange = 1

  for number_of_iteration in range(10):
    next_moves = []
    print( "\n"*2 )
    print( "### Iteración "+ f"{number_of_iteration}" + "###"  )

    routers_intermedio = routers.copy()
    cores_intermedio = cores.copy()

    available_routers = [1 for _ in range(num_nodes)]

    number_of_interchange = 0

    while np.sum(available_routers) :
      identation = " "*4
      level = 0
      print( identation*level +"Particiones Iniciales" )
      print(identation*level + "Routers antes del intercambio:", routers_intermedio)
      print(identation*level + "Núcleos antes del intercambio:", cores_intermedio)
      level = 1
      print(identation*level + "Particion 1" )
      print(identation*level +"Routers antes del intercambio:", [routers_intermedio[i] for i in partition1] )
      print(identation*level +"Núcleos antes del intercambio:", [cores_intermedio[i] for i in partition1])
      print(identation*level +"Particion 2" )
      print(identation*level +"Routers antes del intercambio:", [routers_intermedio[i] for i in partition2])
      print(identation*level +"Núcleos antes del intercambio:", [cores_intermedio[i] for i in partition2])

      cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores_intermedio, routers_intermedio, cost_matrix )

      print( identation*level + "*-" * 20 )
      print( identation*level +"Vision Routers" )
      print( identation*level, cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
      print( identation*level + "--" *20 )
      print(identation*level + "Vision Cores" )
      print( identation*level , cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
      print( identation*level + "*-" * 20 )
      print( "\n" *2 )

      level = 2
      print( identation*level + "# Intercambio #" + f"{number_of_interchange}" )


      difference_cost_vector =  calculate_difference_cost_vector( cost_hop_matrix_routers, partition1, partition2, num_nodes  )
      gain_cost_matrix       =  calculate_gain_cost_matrix( difference_cost_vector, partition1, partition2, None, cost_matrix )
      next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_routers )



      if next_move[0] == None:
        break

      swap =[(next_move[0],next_move[1])]

      cores_intermedio, routers_intermedio = swap_cores_between_routers(cores_intermedio, routers_intermedio, swap )

      swap_routers        = update_swap_nodes( next_move )
      available_routers   = update_available_nodes( available_routers, swap_routers )
      next_moves.append( next_move )

      level = 0
      print( identation*level +"Particiones Intermedias" )
      print(identation*level + "Routers despues del intercambio:", routers_intermedio)
      print(identation*level + "Núcleos despues del intercambio:", cores_intermedio)
      level = 1
      print(identation*level + "Particion 1" )
      print(identation*level +"Routers despues del intercambio:", [routers_intermedio[i] for i in partition1] )
      print(identation*level +"Núcleos despues del intercambio:", [cores_intermedio[i] for i in partition1])
      print(identation*level +"Particion 2" )
      print(identation*level +"Routers despues del intercambio:", [routers_intermedio[i] for i in partition2])
      print(identation*level +"Núcleos despues del intercambio:", [cores_intermedio[i] for i in partition2])


      cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores_intermedio, routers_intermedio, cost_matrix )

      print( identation*level + "*-" * 20 )
      print( identation*level +"Vision Routers" )
      print( identation*level , cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
      print( identation*level + "--" *20 )
      print(identation*level + "Vision Cores" )
      print( identation*level , cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
      print( identation*level + "*-" * 20 )
      print( "\n" *2 )





      number_of_interchange += 1

      #'''
      print("-"*40)
      print("Difference cost Vector")
      print( difference_cost_vector )
      print("-"*40)

      print("\n")

      print("-"*40)
      print("Gain Cost Matrix")
      print( gain_cost_matrix )
      print("-"*40)

      print("\n")

      print("-"*40)
      print("Particiones Potenciales")
      print( "Movimiento Potencial:", next_move )
      print( "Partition1_potencial:", partition1 )
      print( "Partition2_potencial:", partition2 )
      #'''


    max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )

    print(max_partial_sum, partial_next_moves)

    print( "Movimientos Potenciales: ", next_moves )





    print( "Movimientos Finales:", partial_next_moves )


    if max_partial_sum <= 0:
      print("Terminando debido a que max_partial_sum es igual o menor a 0")
      level = 1
      print(identation*level + "Routers antes del intercambio:", routers)
      print(identation*level + "Núcleos antes del intercambio:", cores)
      level = 2
      print( identation*level + "Particion 1" )
      print(identation*level +"Routers antes del intercambio:", [routers[i] for i in partition1] )
      print(identation*level +"Núcleos antes del intercambio:", [cores[i] for i in partition1])
      print(identation*level +"Particion 2" )
      print(identation*level +"Routers antes del intercambio:", [routers[i] for i in partition2])
      print(identation*level +"Núcleos antes del intercambio:", [cores[i] for i in partition2])

      return partition1, partition2


    partition1_post, partition2_post = swap_cores( partial_next_moves, partition1, partition2 )

    level = 0
    print( identation*level +"Particiones Iniciales" )
    print(identation*level + "Routers antes del intercambio:", routers)
    print(identation*level + "Núcleos antes del intercambio:", cores)
    cores, routers = swap_cores_between_routers(cores, routers, partial_next_moves )
    print( identation*level +"Particiones Finales" )
    print(identation*level + "Routers antes del intercambio:", routers)
    print(identation*level + "Núcleos antes del intercambio:", cores)

    cost_hop_matrix_cores, cost_hop_matrix_routers = create_cost_matrix( cores, routers, cost_matrix )

    print( "*-" * 20 )
    print( cost_hop_matrix_routers, np.sum( cost_hop_matrix_routers ) )
    print( "--" *20 )
    print( cost_hop_matrix_cores, np.sum(cost_hop_matrix_cores) )
    print( "*-" * 20 )
    print( "\n" *2 )





    if partition1_post == partition2 and partition2_post == partition1:
      print( "Particiones Finales Iteracion" )
      print( partition1, partition2_post )
      print( partition2, partition1_post )
      return partition1, partition2






In [66]:

routers = [0,1,2,3,4,5]
cores   = [2,1,5,3,4,0]

kernighan_lin( cores, routers, partition1, partition2, cost_matrix )

print(  "\n" )
print( "Particiones Iniciales" + "\t"*2 + "Particiones Finales" )
#print(  f"{partition1}" + "\t"*1 + f"{partition1_kl}" )
#print(  f"{partition2}" + "\t"*1 + f"{partition2_kl}" )




### Iteración 0###
Particiones Iniciales
Routers antes del intercambio: [0, 1, 2, 3, 4, 5]
Núcleos antes del intercambio: [2, 1, 5, 3, 4, 0]
    Particion 1


IndexError: ignored